# Trial Balance Automation - MVP

**Purpose**: Load, validate, and analyze trial balance data

**Author**: Raiden Velarde Guillergan - Data Scientist 

**Date**: November 4, 2025

**Data Source**: `data/raw/Trial Balance/2025/September/`

## Workflow Diagram

```mermaid
flowchart TD
    Start([Start]) --> Init[1. Initialize<br/>Libraries & Logger]
    Init --> LoadFunc[2-3. Define<br/>Loading Functions]
    LoadFunc --> Load[4. Load Data<br/>TB + References]
    Load --> Separate[5. Separate Data]
    Separate --> AddDate[6. Add Date Column]
    AddDate --> Consolidate[7. Consolidate TB]
    Consolidate --> Pivot[8. Create Pivot Table]
    Pivot --> Match[9. Match GL Accounts]
    Match --> CheckNew{New Accounts?}
    CheckNew -->|Yes| Export[Export Updated COA]
    CheckNew -->|No| Done
    Export --> Done([End])
    
    style Start fill:#e1f5e1
    style Done fill:#ffe1e1
    style Pivot fill:#f0e1ff
    style Export fill:#e1f0ff
```

**Note**: Install `Markdown Preview Mermaid Support` extension to view diagrams.  
**Full Documentation**: See `docs/workflow-diagram.md`

# Import library

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
from typing import Union, List, Dict, Any, Hashable, Tuple
from openpyxl import load_workbook
from openpyxl.styles import numbers, Alignment, Font # Import for formatting, alignment, and bolding
from openpyxl.utils import get_column_letter # Import for column letter lookup, needed for merging

import os

warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


### Logger

In [2]:
# Setup logging configuration
import logging

# Get project root for absolute paths
try:
    if '__file__' in globals():
        notebook_dir = Path(__file__).parent
    else:
        notebook_dir = Path.cwd()
        if notebook_dir.name != 'notebooks':
            notebook_dir = notebook_dir / 'notebooks'
    project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
except:
    project_root = Path.cwd()
    if project_root.name == 'notebooks':
        project_root = project_root.parent

# Create logs directory if it doesn't exist
log_dir = project_root / 'logs'
log_dir.mkdir(parents=True, exist_ok=True)

# Create log filename with timestamp
log_filename = f"trial_balance_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
log_path = log_dir / log_filename

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_path),
        logging.StreamHandler()  # Also print to console
    ]
)

logger = logging.getLogger(__name__)

logger.info("="*60)
logger.info("TRIAL BALANCE AUTOMATION - LOGGING INITIALIZED")
logger.info("="*60)
logger.info(f"Project Root: {project_root}")
logger.info(f"Log file: {log_path}")
logger.info(f"Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
logger.info(f"Working directory: {Path.cwd()}")

print(f"\n✓ Logging configured successfully")
print(f"📂 Project Root: {project_root}")
print(f"📝 Log file: {log_path}")


2025-11-21 13:25:28,981 - INFO - ============================================================
2025-11-21 13:25:28,983 - INFO - TRIAL BALANCE AUTOMATION - LOGGING INITIALIZED
2025-11-21 13:25:28,983 - INFO - TRIAL BALANCE AUTOMATION - LOGGING INITIALIZED
2025-11-21 13:25:28,984 - INFO - ============================================================
2025-11-21 13:25:28,984 - INFO - Project Root: d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance
2025-11-21 13:25:28,985 - INFO - Log file: d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\logs\trial_balance_20251121_132528.log
2025-11-21 13:25:28,986 - INFO - Session started: 2025-11-21 13:25:28
2025-11-21 13:25:28,987 - INFO - Working directory: d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\notebooks



✓ Logging configured successfully
📂 Project Root: d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance
📝 Log file: d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\logs\trial_balance_20251121_132528.log


### Parameters (for Papermill)

These parameters can be overridden when running via papermill/GUI.

In [3]:
# Parameters - can be overridden by papermill
# Tag this cell with 'parameters' for papermill to inject values
# NOTE: These are NOT used anymore - config file approach is used instead
year = None  # Deprecated - use config file
month = None  # Deprecated - use config file

logger.info("📝 Parameters cell (deprecated - using config file approach instead)")


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4dd' in position 33: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\.venv\Lib\site-packages\traitlets\config\application.p

## 1. Setup and Configuration

## 2. Data Loading Function

## 3. Reference Data Loading Function

In [4]:
def load_reference_data(base_path=None):
    """
    Load reference data (COA Mapping and Portfolio Mapping) from the latest files.
    Supports both CSV and XLSX file formats.
    
    Args:
        base_path: Optional path to references folder. If None, uses project_root/data/references
    
    Returns:
        dict: Dictionary containing:
            - 'coa_mapping': DataFrame from COA Mapping folder (latest file)
            - 'portfolio_mapping': DataFrame from Portfolio Mapping folder (latest file)
            - 'metadata': dict with loading information
    """
    
    # Use absolute path from project root if not provided
    if base_path is None:
        base_path = project_root / 'data' / 'references'
    else:
        base_path = Path(base_path)
    
    # Initialize result dictionary
    result = {
        'coa_mapping': None,
        'portfolio_mapping': None,
        'metadata': {
            'load_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'coa_mapping_file': None,
            'portfolio_mapping_file': None
        }
    }
    
    # Helper function to load file (CSV or XLSX)
    def load_file(file_path):
        if file_path.suffix.lower() == '.csv':
            return pd.read_csv(file_path)
        elif file_path.suffix.lower() in ['.xlsx', '.xls']:
            return pd.read_excel(file_path)
        else:
            raise ValueError(f"Unsupported file format: {file_path.suffix}")
    
    # Define folder paths
    coa_mapping_folder = base_path / 'COA Mapping'
    portfolio_mapping_folder = base_path / 'Portfolio Mapping'
    
    # ========== Load COA Mapping (Latest File) ==========
    if coa_mapping_folder.exists():
        logger.info(f"📂 Loading COA Mapping from: {coa_mapping_folder}")
        
        # Get all CSV and XLSX files sorted by modification time (latest first)
        files = sorted(
            list(coa_mapping_folder.glob('*.csv')) + 
            list(coa_mapping_folder.glob('*.xlsx')) + 
            list(coa_mapping_folder.glob('*.xls')),
            key=lambda f: f.stat().st_mtime, 
            reverse=True
        )
        
        if not files:
            logger.warning(f"  ⚠️  WARNING: No CSV or XLSX files found in {coa_mapping_folder}")
        else:
            latest_file = files[0]
            result['coa_mapping'] = load_file(latest_file)
            result['metadata']['coa_mapping_file'] = latest_file.name
            
            logger.info(f"  ✓ Loaded latest file: {latest_file.name}")
            logger.info(f"    Records: {len(result['coa_mapping'])}")
            
            if len(files) > 1:
                logger.info(f"    Note: {len(files)} files found, loaded the most recent")
    else:
        logger.warning(f"⚠️  WARNING: COA Mapping folder not found: {coa_mapping_folder}")
    
    # ========== Load Portfolio Mapping (Latest File) ==========
    if portfolio_mapping_folder.exists():
        logger.info(f"\n📂 Loading Portfolio Mapping from: {portfolio_mapping_folder}")
        
        # Get all CSV and XLSX files sorted by modification time (latest first)
        files = sorted(
            list(portfolio_mapping_folder.glob('*.csv')) + 
            list(portfolio_mapping_folder.glob('*.xlsx')) + 
            list(portfolio_mapping_folder.glob('*.xls')),
            key=lambda f: f.stat().st_mtime, 
            reverse=True
        )
        
        if not files:
            logger.warning(f"  ⚠️  WARNING: No CSV or XLSX files found in {portfolio_mapping_folder}")
        else:
            latest_file = files[0]
            result['portfolio_mapping'] = load_file(latest_file)
            result['metadata']['portfolio_mapping_file'] = latest_file.name
            
            logger.info(f"  ✓ Loaded latest file: {latest_file.name}")
            logger.info(f"    Records: {len(result['portfolio_mapping'])}")
            
            if len(files) > 1:
                logger.info(f"    Note: {len(files)} files found, loaded the most recent")
    else:
        logger.warning(f"⚠️  WARNING: Portfolio Mapping folder not found: {portfolio_mapping_folder}")
    
    return result


In [5]:
# Load Data with Config + Fallback
import json
from pathlib import Path

logger.info("="*60)
logger.info("LOADING TRIAL BALANCE DATA")
logger.info("="*60)

# Define paths - Get absolute path to project root first
# When running via papermill, __file__ is available
# When running in Jupyter, use Path.cwd() and navigate up
try:
    # Try to get notebook's actual location
    if '__file__' in globals():
        notebook_dir = Path(__file__).parent
    else:
        # In Jupyter/interactive mode, assume we're in notebooks/
        notebook_dir = Path.cwd()
        if notebook_dir.name != 'notebooks':
            notebook_dir = notebook_dir / 'notebooks'
    
    # Project root is parent of notebooks directory
    if notebook_dir.name == 'notebooks':
        project_root = notebook_dir.parent
    else:
        project_root = notebook_dir
        
except:
    # Fallback: use current directory
    project_root = Path.cwd()
    if project_root.name == 'notebooks':
        project_root = project_root.parent

# Construct absolute paths
config_path = project_root / 'config' / 'run_config.json'
default_raw_path = project_root / 'data' / 'raw' / 'Trial Balance'

logger.info(f"📂 Project Root: {project_root}")
logger.info(f"📂 Config Path: {config_path}")
logger.info(f"📂 Default Raw Path: {default_raw_path}")

# Initialize variables
data_path = None
year = None
month = None
config_source = None

# ========== STRATEGY 1: Try Config File (User Selection) ==========
if config_path.exists():
    logger.info(f"📝 Found config file: {config_path}")
    
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
        
        data_path = Path(config.get('data_path', ''))
        year = config.get('year')
        month = config.get('month')
        
        logger.info(f"✅ Config loaded successfully")
        logger.info(f"   Year: {year}")
        logger.info(f"   Month: {month}")
        logger.info(f"   Data Path: {data_path}")
        
        # Validate the path exists
        if data_path.exists():
            config_source = "user_selection"
            logger.info(f"✅ Data folder validated: {data_path}")
        else:
            logger.warning(f"⚠️  Config path does not exist: {data_path}")
            logger.warning(f"   Will try fallback methods...")
            data_path = None
            
    except json.JSONDecodeError as e:
        logger.error(f"❌ Config file is corrupted: {e}")
        logger.warning(f"   Will try fallback methods...")
    except Exception as e:
        logger.error(f"❌ Error reading config: {e}")
        logger.warning(f"   Will try fallback methods...")
else:
    logger.warning(f"⚠️  Config file not found: {config_path}")
    logger.info(f"   This is normal for direct notebook execution")

# ========== STRATEGY 2: Auto-Detect Latest (Fallback) ==========
if data_path is None:
    logger.info("\n" + "="*60)
    logger.info("FALLBACK: AUTO-DETECTING LATEST DATA FOLDER")
    logger.info("="*60)
    
    if not default_raw_path.exists():
        error_msg = f"❌ FATAL: Raw data folder not found: {default_raw_path}"
        logger.error(error_msg)
        logger.error(f"   Project Root: {project_root}")
        logger.error(f"   Please check your project structure!")
        logger.error(f"   Current working directory: {Path.cwd()}")
        raise FileNotFoundError(error_msg)
    
    # Find latest year (only numeric folders)
    year_folders = sorted(
        [f for f in default_raw_path.iterdir() if f.is_dir() and f.name.isdigit()],
        reverse=True
    )
    
    if not year_folders:
        error_msg = f"❌ FATAL: No year folders found in {default_raw_path}"
        logger.error(error_msg)
        logger.error(f"   Available folders: {[f.name for f in default_raw_path.iterdir() if f.is_dir()]}")
        raise FileNotFoundError(error_msg)
    
    latest_year_folder = year_folders[0]
    year = latest_year_folder.name
    logger.info(f"📅 Latest year found: {year}")
    
    # Find latest month (by modification time)
    month_folders = sorted(
        [f for f in latest_year_folder.iterdir() if f.is_dir()],
        key=lambda x: x.stat().st_mtime,
        reverse=True
    )
    
    if not month_folders:
        error_msg = f"❌ FATAL: No month folders found in {latest_year_folder}"
        logger.error(error_msg)
        raise FileNotFoundError(error_msg)
    
    latest_month_folder = month_folders[0]
    month = latest_month_folder.name
    data_path = latest_month_folder
    config_source = "auto_detect"
    
    logger.info(f"📅 Latest month found: {month}")
    logger.info(f"✅ Auto-detected path: {data_path}")

# ========== FINAL VALIDATION ==========
logger.info("\n" + "="*60)
logger.info("DATA LOADING SUMMARY")
logger.info("="*60)
logger.info(f"Source: {config_source.upper().replace('_', ' ')}")
logger.info(f"Year: {year}")
logger.info(f"Month: {month}")
logger.info(f"Data Path: {data_path}")

# Validate required subfolders exist
tb_folder = data_path / 'Trial Balance'
coa_folder = data_path / 'Chart of Accounts'

if not tb_folder.exists():
    logger.error(f"❌ Trial Balance folder missing: {tb_folder}")
    raise FileNotFoundError(f"Required folder not found: {tb_folder}")
    
if not coa_folder.exists():
    logger.warning(f"⚠️  Chart of Accounts folder missing: {coa_folder}")

# ========== LOAD DATA ==========
try:
    # Load trial balance data using the determined path
    data = {
        'trial_balance': {},
        'chart_of_accounts': None,
        'metadata': {
            'year': year,
            'month': month,
            'load_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'tb_files': []
        }
    }
    
    # Load Trial Balance CSVs
    logger.info(f"\n📊 Loading Trial Balance files from: {tb_folder}")
    for file in sorted(tb_folder.glob("*.csv")):
        try:
            file_date = datetime.strptime(file.stem, "%m-%d-%Y")
            date_key = file_date.strftime("%Y-%m-%d")
            data['trial_balance'][date_key] = pd.read_csv(file)
            data['metadata']['tb_files'].append({
                'filename': file.name,
                'date': date_key,
                'records': len(data['trial_balance'][date_key])
            })
            logger.info(f"  ✅ {file.name}: {len(data['trial_balance'][date_key])} records")
        except ValueError:
            logger.warning(f"  ⚠️  Skipped non-standard filename: {file.name}")
    
    # Load Chart of Accounts
    coa_files = list(coa_folder.glob("*.csv"))
    if coa_files:
        logger.info(f"\n📋 Loading Chart of Accounts from: {coa_folder}")
        data['chart_of_accounts'] = pd.read_csv(coa_files[0])
        logger.info(f"  ✅ {coa_files[0].name}: {len(data['chart_of_accounts'])} records")
    
    logger.info("\n" + "="*60)
    logger.info("✅ DATA LOADED SUCCESSFULLY")
    logger.info("="*60)
    logger.info(f"Trial Balance Files: {len(data['trial_balance'])}")
    logger.info(f"Chart of Accounts: {'Loaded' if data['chart_of_accounts'] is not None else 'Not Loaded'}")
    
    if data['trial_balance']:
        dates = list(data['trial_balance'].keys())
        logger.info(f"Date Range: {min(dates)} to {max(dates)}")
    
    print("\n✅ Data loading complete!")
    
except Exception as e:
    logger.error(f"❌ FATAL ERROR during data loading: {e}")
    logger.error(f"   Data Path Attempted: {data_path}")
    logger.error(f"   TB Folder: {tb_folder}")
    logger.error(f"   TB Folder Exists: {tb_folder.exists()}")
    raise

# ========== EXTRACT TO EXPECTED VARIABLE NAMES ==========
# The rest of the notebook expects these variable names
trial_balance_data = data['trial_balance']
chart_of_accounts = data['chart_of_accounts']

logger.info("\n✅ Variables ready:")
logger.info(f"   - trial_balance_data: {len(trial_balance_data)} dates")
logger.info(f"   - chart_of_accounts: {len(chart_of_accounts) if chart_of_accounts is not None else 0} records")


2025-11-21 13:25:29,066 - INFO - ============================================================
2025-11-21 13:25:29,067 - INFO - LOADING TRIAL BALANCE DATA
2025-11-21 13:25:29,068 - INFO - ============================================================
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4c2' in position 33: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\UserProfile\Documents\@ VFC\pemi


✅ Data loading complete!


## 4. Load Data

## 4A. Interactive Folder Selection UI

Select the year and month folder to load trial balance data from.

In [6]:
# DEPRECATED: Interactive Folder Selector UI
# This cell is disabled when running via papermill/GUI
# The GUI now writes a config file that Cell 13 reads instead

print("⚠️  This interactive folder selector is DISABLED when running via GUI/papermill")
print("   The system now uses config file approach (config/run_config.json)")
print("   See Cell 13 for the automatic config-based data loading")


⚠️  This interactive folder selector is DISABLED when running via GUI/papermill
   The system now uses config file approach (config/run_config.json)
   See Cell 13 for the automatic config-based data loading


In [7]:
# ============================================================
# ⚠️  DEPRECATED FUNCTION - DO NOT USE
# ============================================================
# Function: load_trial_balance_data_from_selection()
# 
# REASON FOR DEPRECATION:
# - Relied on 'selected_folder' dict from interactive UI widgets
# - Interactive widgets don't work with papermill execution
# 
# REPLACED BY:
# - Cell 13: Reads config/run_config.json (written by GUI)
# - Cell 13: Has automatic fallback to latest year/month
# - Cell 13: Uses absolute paths (works in any execution context)
#
# This function definition is kept for reference but should not be called.
# ============================================================

logger.info("⚠️  Cell 17 (load_trial_balance_data_from_selection function) is DEPRECATED")
logger.info("   Function not defined - use Cell 13 data loading instead")


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode characters in position 33-34: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\.venv\Lib\site-packages\traitlets\config\application.py", line 

In [8]:
# ============================================================
# ⚠️  DEPRECATED CELL - DO NOT USE
# ============================================================
# This cell used the old 'selected_folder' approach with interactive UI
# 
# REASON FOR DEPRECATION:
# - Interactive widgets don't work with papermill
# - GUI now handles folder selection and writes config file
# 
# NEW APPROACH:
# - Cell 13 reads config/run_config.json (written by GUI)
# - Cell 13 has automatic fallback to latest year/month if no config
# - All paths are absolute (works with papermill execution context)
#
# IF YOU NEED TO LOAD DATA:
# - Use Cell 13 (reads config file and loads data automatically)
# - Or run via GUI: scripts/launchers/launch_gui.bat
# ============================================================

logger.info("⚠️  Cell 18 (selected_folder loader) is DEPRECATED - skipping")
logger.info("   Data loading happens in Cell 13 using config file approach")


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode characters in position 33-34: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\.venv\Lib\site-packages\traitlets\config\application.py", line 

In [9]:
# Load reference data
reference_data = load_reference_data()

print("\n" + "="*60)
print("📋 REFERENCE DATA LOADING SUMMARY")
print("="*60)
print(f"Load Time: {reference_data['metadata']['load_timestamp']}")
print(f"\nCOA Mapping: {'Loaded' if reference_data['coa_mapping'] is not None else 'Not Loaded'}")
if reference_data['metadata']['coa_mapping_file']:
    print(f"  File: {reference_data['metadata']['coa_mapping_file']}")
print(f"\nPortfolio Mapping: {'Loaded' if reference_data['portfolio_mapping'] is not None else 'Not Loaded'}")
if reference_data['metadata']['portfolio_mapping_file']:
    print(f"  File: {reference_data['metadata']['portfolio_mapping_file']}")
print("\n" + "="*60)

# Extract to expected variable names (for backward compatibility)
coa_mapping = reference_data['coa_mapping']
portfolio_mapping = reference_data['portfolio_mapping']

logger.info(f"✅ Reference variables ready:")
logger.info(f"   - coa_mapping: {len(coa_mapping) if coa_mapping is not None else 0} records")
logger.info(f"   - portfolio_mapping: {len(portfolio_mapping) if portfolio_mapping is not None else 0} records")


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4c2' in position 33: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\.venv\Lib\site-packages\traitlets\config\application.p


📋 REFERENCE DATA LOADING SUMMARY
Load Time: 2025-11-21 13:25:29

COA Mapping: Loaded
  File: Chart of Accounts Mapping as of 10.10.2025.xlsx

Portfolio Mapping: Loaded
  File: PEMI_Account_name_porfolio_mapping.xlsx



In [10]:
reference_data.keys()

dict_keys(['coa_mapping', 'portfolio_mapping', 'metadata'])

## 5. Separate Data by Source

In [11]:
# DEPRECATED: Manual folder selection trigger
# This cell is disabled when running via papermill/GUI

print("⚠️  Manual folder selection is DISABLED")
print("   Data loading now happens automatically in Cell 13")
print("   using the config file from the GUI (config/run_config.json)")


⚠️  Manual folder selection is DISABLED
   Data loading now happens automatically in Cell 13
   using the config file from the GUI (config/run_config.json)


## 6. Add Date Column to Trial Balance Data

In [12]:
# Add 'Date' column to each Trial Balance DataFrame
for date_key, df in trial_balance_data.items():
    df['Date'] = date_key

print("✓ Date column added to all Trial Balance DataFrames")
print(f"\nProcessed {len(trial_balance_data)} date(s)")

✓ Date column added to all Trial Balance DataFrames

Processed 30 date(s)


## 7. Consolidate Trial Balance Data

In [13]:
# Consolidate all Trial Balance DataFrames into a single DataFrame
trial_balance_consolidated = pd.concat(trial_balance_data.values(), ignore_index=True)

print("✓ Trial Balance data consolidated")
print(f"\nTotal records: {len(trial_balance_consolidated):,}")
print(f"Date range: {trial_balance_consolidated['Date'].min()} to {trial_balance_consolidated['Date'].max()}")
print(f"Unique dates: {trial_balance_consolidated['Date'].nunique()}")
print(f"\nColumns: {trial_balance_consolidated.columns.tolist()}")

✓ Trial Balance data consolidated

Total records: 40,162
Date range: 2025-09-01 to 2025-09-30
Unique dates: 30

Columns: ['bookname', 'level1accountname', 'level2accountname', 'accountname', 'Portfolio', 'Portcode ', 'Ext. Portfolio Code', 'Ext. Portcode 2', 'openingdebit', 'openingcredit', 'perioddebit', 'periodcredit', 'closingdebit', 'closingcredit', 'netamt', 'GL Code', 'cust_accountid', 'Account ID', 'accounttype', 'accountsubtype', 'descr', 'Entity', 'Date']


In [14]:
# len(trial_balance_consolidated['Date'].unique())

trial_balance_consolidated

,bookname,level1accountname,level2accountname,accountname,Portfolio,Portcode,Ext. Portfolio Code,Ext. Portcode 2,openingdebit,openingcredit,perioddebit,periodcredit,closingdebit,closingcredit,netamt,GL Code,cust_accountid,Account ID,accounttype,accountsubtype,descr,Entity,Date
0,MAIN ACCOUNT,Bangko Sentral Ng Pilipinas Provident Fund (bs...,Asset,A/R - Others PHP,Bangko Sentral Ng Pilipinas Provident Fund (bs...,607896,NaN,NaN,1008.49,1008.49,0.00,0.00,1008.49,1008.49,0.00,NaN,16204,16204,B/S,A,INVESTMENT,NaN,2025-09-01
1,MAIN ACCOUNT,Bangko Sentral Ng Pilipinas Provident Fund (bs...,Asset,ACCRUED INTEREST RECEIVABLE,Bangko Sentral Ng Pilipinas Provident Fund (bs...,607896,NaN,NaN,106763.37,106763.37,0.00,0.00,106763.37,106763.37,0.00,NaN,16149,16149,B/S,A,INVESTMENT,NaN,2025-09-01
2,MAIN ACCOUNT,Bangko Sentral Ng Pilipinas Provident Fund (bs...,Asset,AIR - MMP - PHP,Bangko Sentral Ng Pilipinas Provident Fund (bs...,607896,NaN,NaN,209773.51,209773.51,3228.33,0.00,213001.84,209773.51,3228.33,NaN,16369,16369,B/S,A,INVESTMENT,NaN,2025-09-01
3,MAIN ACCOUNT,Bangko Sentral Ng Pilipinas Provident Fund (bs...,Asset,DUE FROM BROKERS,Bangko Sentral Ng Pilipinas Provident Fund (bs...,607896,NaN,NaN,54701864.23,54701864.23,3629042.67,0.00,58330906.90,54701864.23,3629042.67,NaN,16367,16367,B/S,A,INVESTMENT,NaN,2025-09-01
4,MAIN ACCOUNT,Bangko Sentral Ng Pilipinas Provident Fund (bs...,Asset,Dividend Receivable,Bangko Sentral Ng Pilipinas Provident Fund (bs...,607896,NaN,NaN,15387963.51,12854473.67,0.00,0.00,15387963.51,12854473.67,2533489.84,NaN,16320,16320,B/S,A,INVESTMENT,NaN,2025-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40157,MAIN ACCOUNT,Social Security System Ima (sss)-607900,Income,Int. Income - CASA Bank Deposits-Tax-Pd (net)-PhP,Social Security System Ima (sss)-607900,607900,NaN,NaN,0.00,9319.86,0.00,0.00,0.00,9319.86,-9319.86,NaN,16895,16895,P/L,I,INCOME,NaN,2025-09-30
40158,MAIN ACCOUNT,Social Security System Ima (sss)-607900,Income,Trading Gain / Loss,Social Security System Ima (sss)-607900,607900,NaN,NaN,1398560.00,11153168.26,0.00,0.00,1398560.00,11153168.26,-9754608.26,NaN,17019,17019,P/L,E,EXPENSE,NaN,2025-09-30
40159,MAIN ACCOUNT,Social Security System Ima (sss)-607900,Income,Unrealized Gain / Loss,Social Security System Ima (sss)-607900,607900,NaN,NaN,74807865735.52,74773552443.83,335201253.77,325544964.47,75143066989.29,75099097408.30,43969580.99,NaN,17017,17017,P/L,I,INCOME,NaN,2025-09-30
40160,MAIN ACCOUNT,Social Security System Ima (sss)-607900,Liability,A/P - Other,Social Security System Ima (sss)-607900,607900,NaN,NaN,9314935.00,9314935.00,0.00,0.00,9314935.00,9314935.00,0.00,NaN,16774,16774,B/S,L,CURRENT LIABILITIES,NaN,2025-09-30


### `GET MONTH AND LATEST DAY VALUE`

In [15]:
def get_latest_day_per_month(df: pd.DataFrame, date_col: str = 'Date', flat_output: bool = False) -> Union[Dict[str, Dict[str, Any]], Dict[str, str]]:
    """
    Processes a DataFrame to find the latest date for each unique reporting month.

    If flat_output is False (default), it returns a structured, nested dictionary 
    with a unique key ('YYYY-Month Name').
    
    If flat_output is True, it returns a flat dictionary mapping 'Month Name' to 
    the 'Latest Date'. WARNING: This flat output is NOT robust for multi-year data
    as it will overwrite older months with newer ones.

    Args:
        df: The input DataFrame (e.g., trial_balance_consolidated).
        date_col: The name of the date column.
        flat_output: If True, returns a simple {'Month Name': 'YYYY-MM-DD'} map, 
                     losing year context. Defaults to False.

    Returns:
        The requested dictionary format.
    """
    if df.empty or date_col not in df.columns:
        print(f"Error: DataFrame is empty or does not contain a '{date_col}' column.")
        return {}

    # 1. Ensure the date column is in datetime format and drop invalid rows
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df = df.dropna(subset=[date_col])

    # 2. Group by Year and Month (robust grouping) and find the maximum date in each group.
    df_latest_dates = df.groupby([df[date_col].dt.year, df[date_col].dt.month])[date_col].max()

    # --- 3. Format the result based on flat_output parameter ---
    
    if flat_output:
        # User requested the simple, flat format (Interpretation A)
        latest_dates_flat: Dict[str, str] = {}
        for (year, month), latest_date in df_latest_dates.items():
            # Uses the month name as the key, which is ambiguous across years
            month_name = latest_date.strftime('%B')
            latest_dates_flat[month_name] = latest_date.strftime('%Y-%m-%d')
        
        # WARNING: The loop inherently takes the LATEST date found for that month name across ALL years.
        return latest_dates_flat
        
    else:
        # Default: Structured, unique-key output (Robust)
        latest_dates_structured: Dict[str, Dict[str, Any]] = {}

        for (year, month), latest_date in df_latest_dates.items():
            # Create a unique primary key: 'YYYY-Month Name'
            primary_key = latest_date.strftime('%Y-%B')
            
            # Create the structured record containing all required keys
            latest_dates_structured[primary_key] = {
                'date': latest_date.strftime('%Y-%m-%d'),
                'month_name': latest_date.strftime('%B'),
                'month_num': int(month),
                'year': int(year)
            }

        return latest_dates_structured

In [16]:
# 1. Robust, Structured Output (Default)
latest_reporting_dates_structured = get_latest_day_per_month(trial_balance_consolidated, flat_output=False) # JSON 

# 2. Flat Output (As requested, but with data integrity warning)
latest_reporting_dates_flat = get_latest_day_per_month(trial_balance_consolidated, flat_output=True) # DICT

# latest_reporting_dates_flat
latest_reporting_dates_structured

{'2025-September': {'date': '2025-09-30',
  'month_name': 'September',
  'month_num': 9,
  'year': 2025}}

## 8. Create Pivot Table

In [17]:
# Create pivot table
trial_balance_pivot_table = trial_balance_consolidated.pivot_table(
    index='accountname',           # Rows: GL Account
    columns='level1accountname',   # Columns: Fund Name
    values='netamt',               # Values: Balance
    aggfunc='sum',                 # Sum the netamt
    fill_value=0                   # Fill missing values with 0
)

# Rename index and columns for clarity
trial_balance_pivot_table.index.name = 'GL Account'
trial_balance_pivot_table.columns.name = 'Fund Name'

print("✓ Pivot table created")
print(f"\nShape: {trial_balance_pivot_table.shape[0]} GL Accounts × {trial_balance_pivot_table.shape[1]} Funds")
print(f"Total Balance: {trial_balance_pivot_table.sum().sum():,.2f}")

# Display pivot table
trial_balance_pivot_table

✓ Pivot table created

Shape: 313 GL Accounts × 16 Funds
Total Balance: -0.25
Total Balance: -0.25


Fund Name,Bangko Sentral Ng Pilipinas Provident Fund (bsppf)-607896,"De La Salle - College Of Saint Benilde, Inc. (dlsu-csb)-607894","De La Salle University, Inc. (dlsu)-607892",GOVERNMENT SERVICE INSURANCE SYSTEM-500032,Government Service Insurance System Ima (gsis)-607898,PHILEQUITY DIVIDEND YIELD FUND,PHILEQUITY DOLLAR INCOME FUND,"PHILEQUITY FUND, INC.","PHILEQUITY MANAGEMENT, INC.",PHILEQUITY PESO BOND FUND,PHILEQUITY PSE INDEX FUND,"Philequity Alpha One Fund, Inc.","Philequity Alpha One Fund, Inc. s","Philequity MSCI Philippines Index Fund, Inc.",Philequity Mgt Inc. Ima#008 - 85399-607902,Social Security System Ima (sss)-607900
GL Account,,,,,,,,,,,,,,,,
ANNUAL FEE - DIRECTOR,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Annual Fee - Director,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BIR Registration/License Fee,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-3000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Business Permit,0.00,0.00,0.00,0.00,0.00,0.00,-25.50,-11283543.05,0.00,-0.13,0.00,0.00,0.00,0.00,0.00,0.00
Doc. Stamp Tax,0.00,0.00,0.00,0.00,0.00,-3373495.28,-0.00,-1358391.92,0.00,-0.12,-0.03,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Union Bank of the Philippines, Inc.-6101011305",0.00,0.00,0.00,0.00,0.00,0.00,1974222.28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Unrealized Gain / Loss,-55505407.93,14759960.46,-43211680.26,0.00,58149555.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,92292705543.68,77587036.34
Vat Output - Others,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-6253161246.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [18]:
list(trial_balance_pivot_table.columns)

['Bangko Sentral Ng Pilipinas Provident Fund (bsppf)-607896',
 'De La Salle - College Of Saint Benilde, Inc. (dlsu-csb)-607894',
 'De La Salle University, Inc. (dlsu)-607892',
 'GOVERNMENT SERVICE INSURANCE SYSTEM-500032',
 'Government Service Insurance System Ima (gsis)-607898',
 'PHILEQUITY DIVIDEND YIELD FUND',
 'PHILEQUITY DOLLAR INCOME FUND',
 'PHILEQUITY FUND, INC.',
 'PHILEQUITY MANAGEMENT, INC.',
 'PHILEQUITY PESO BOND FUND',
 'PHILEQUITY PSE INDEX FUND',
 'Philequity Alpha One Fund, Inc.',
 'Philequity Alpha One Fund, Inc. s',
 'Philequity MSCI Philippines Index Fund, Inc.',
 'Philequity Mgt Inc. Ima#008 - 85399-607902',
 'Social Security System Ima (sss)-607900']

In [19]:
def capitalize_pivot_columns(pivot_df):
    """
    Capitalize all column names in the pivot table except for the index ('GL Account').
    
    Parameters:
        pivot_df (DataFrame): The pivot table with Fund Names as columns
    
    Returns:
        DataFrame: Pivot table with capitalized column names
    """
    # Create a copy to avoid modifying original
    df_copy = pivot_df.copy()
    
    # Capitalize all column names
    df_copy.columns = [col.upper() for col in df_copy.columns]
    
    print("✓ Pivot table columns capitalized")
    print(f"  Columns: {list(df_copy.columns)}")
    
    return df_copy

# Apply the function to the pivot table
trial_balance_pivot_table = capitalize_pivot_columns(trial_balance_pivot_table)

# Display updated columns
print(f"\n📋 Updated columns: {list(trial_balance_pivot_table.columns)}")


✓ Pivot table columns capitalized
  Columns: ['BANGKO SENTRAL NG PILIPINAS PROVIDENT FUND (BSPPF)-607896', 'DE LA SALLE - COLLEGE OF SAINT BENILDE, INC. (DLSU-CSB)-607894', 'DE LA SALLE UNIVERSITY, INC. (DLSU)-607892', 'GOVERNMENT SERVICE INSURANCE SYSTEM-500032', 'GOVERNMENT SERVICE INSURANCE SYSTEM IMA (GSIS)-607898', 'PHILEQUITY DIVIDEND YIELD FUND', 'PHILEQUITY DOLLAR INCOME FUND', 'PHILEQUITY FUND, INC.', 'PHILEQUITY MANAGEMENT, INC.', 'PHILEQUITY PESO BOND FUND', 'PHILEQUITY PSE INDEX FUND', 'PHILEQUITY ALPHA ONE FUND, INC.', 'PHILEQUITY ALPHA ONE FUND, INC. S', 'PHILEQUITY MSCI PHILIPPINES INDEX FUND, INC.', 'PHILEQUITY MGT INC. IMA#008 - 85399-607902', 'SOCIAL SECURITY SYSTEM IMA (SSS)-607900']

📋 Updated columns: ['BANGKO SENTRAL NG PILIPINAS PROVIDENT FUND (BSPPF)-607896', 'DE LA SALLE - COLLEGE OF SAINT BENILDE, INC. (DLSU-CSB)-607894', 'DE LA SALLE UNIVERSITY, INC. (DLSU)-607892', 'GOVERNMENT SERVICE INSURANCE SYSTEM-500032', 'GOVERNMENT SERVICE INSURANCE SYSTEM IMA (GSIS)-

# `Chart of Accounts`

In [20]:
chart_of_accounts

,book,Account ID,accountname,level1accountname,level2accountname,levels,include_for_aum,accounttype,accountsubtype,Portcode,currencycode,active_status,Authorisation Status,Authorised By,Authorised On,createdby,Deleted,ac_type,gl_code,template_id
0,MAIN ACCOUNT,1329,Business Permit,GOVERNMENT SERVICE INSURANCE SYSTEM-500032,Expense,3,NO,P/L,EXPENSE,500032.00,PHP,Yes,AUTHORISED,IT,07-02-2016 12:13:46,TEST1,N,G/L Account,96-00000,256.00
1,MAIN ACCOUNT,1330,SEC Registration/License Fee,GOVERNMENT SERVICE INSURANCE SYSTEM-500032,Expense,3,NO,P/L,EXPENSE,500032.00,PHP,Yes,AUTHORISED,TEST1,07-01-2016 00:00:00,TEST1,N,G/L Account,96-00000,67.00
2,MAIN ACCOUNT,1331,BIR Registration/License Fee,GOVERNMENT SERVICE INSURANCE SYSTEM-500032,Expense,3,NO,P/L,EXPENSE,500032.00,PHP,Yes,AUTHORISED,TEST1,07-01-2016 00:00:00,TEST1,N,G/L Account,96-00000,257.00
3,MAIN ACCOUNT,1332,Doc. Stamp Tax,GOVERNMENT SERVICE INSURANCE SYSTEM-500032,Expense,3,NO,P/L,EXPENSE,500032.00,PHP,Yes,AUTHORISED,TEST1,07-01-2016 00:00:00,TEST1,N,G/L Account,96-00000,258.00
4,MAIN ACCOUNT,1333,Community TAx Cert (CTC),GOVERNMENT SERVICE INSURANCE SYSTEM-500032,Expense,3,NO,P/L,EXPENSE,500032.00,PHP,Yes,AUTHORISED,TEST1,07-01-2016 00:00:00,TEST1,N,G/L Account,96-00000,68.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3835,MAIN ACCOUNT,16916,East West Bank-Tektite-200021011982,"De La Salle - College Of Saint Benilde, Inc. (...",Asset,3,NO,B/S,CURRENT ASSET,607894.00,PHP,Yes,AUTHORISED,mmagcamit,01-31-2025 21:22:38,CREDENCE,N,NaN,NaN,NaN
3836,MAIN ACCOUNT,16917,East West Bank-Tektite-200021012048,"De La Salle University, Inc. (dlsu)-607892",Asset,3,NO,B/S,CURRENT ASSET,607892.00,PHP,Yes,AUTHORISED,mmagcamit,01-31-2025 21:23:08,CREDENCE,N,NaN,NaN,NaN
3837,MAIN ACCOUNT,16918,East West Bank-Tektite-200005350106,Government Service Insurance System Ima (gsis)...,Asset,3,NO,B/S,CURRENT ASSET,607898.00,PHP,Yes,AUTHORISED,mmagcamit,01-31-2025 21:23:43,CREDENCE,N,NaN,NaN,NaN
3838,MAIN ACCOUNT,16919,Land Bank of the Philippines-The Luxe-3902112501,Social Security System Ima (sss)-607900,Asset,3,NO,B/S,CURRENT ASSET,607900.00,PHP,No,UNAUTHORISED,NaN,NaN,CREDENCE,N,NaN,NaN,NaN


In [21]:
def create_unique_column_df(
    df: pd.DataFrame, 
    col_name: str = 'accountname',
    sort_ascending: bool = True
) -> pd.DataFrame:
    """
    Creates a new DataFrame containing only the specified column, with all 
    duplicate values removed, and sorts the result.

    Args:
        df: The input pandas DataFrame.
        col_name: The name of the column to isolate, deduplicate, and sort. 
                  Defaults to 'accountname'.
        sort_ascending: If True, sorts the unique values in ascending order 
                        (A-Z or 0-9). If False, sorts descending. Defaults to True.

    Returns:
        A new DataFrame with a single column (col_name) containing only unique values, sorted.
    """
    if col_name not in df.columns:
        print(f"Error: Column '{col_name}' not found in the DataFrame.")
        return pd.DataFrame()

    # 1. Select the single column and create a copy
    single_col_df = df[[col_name]].copy()

    # 2. Drop duplicate rows based on that column
    unique_df = single_col_df.drop_duplicates().reset_index(drop=True)
    
    # 3. Sort the unique values (NEW STEP)
    # The sort_values method is applied directly to the resulting unique DataFrame.
    unique_df = unique_df.sort_values(by=col_name, ascending=sort_ascending).reset_index(drop=True)

    return unique_df

In [22]:
# 1. Your current request: Unique 'accountname' values in a new DataFrame
unique_account_df = create_unique_column_df(chart_of_accounts, col_name='accountname')
unique_account_df

,accountname
0,ANNUAL FEE - DIRECTOR
1,Annual Fee - Director
2,BIR Registration/License Fee
3,Business Permit
4,Doc. Stamp Tax
...,...
504,Unrealized Gains/(Losses) - Foreign Currency
505,Vat Output - Others
506,Withdrawal-Interest
507,Withdrawal-Principal


In [23]:
coa_mapping

,GL Account,TB Account Name,Account Type,FS Classification
0,Bank of the Philippine Islands (BPI) - The For...,Cash in Bank,Asset,Cash and Cash Equivalents
1,Bank of the Philippine Islands-004011022923,Cash in Bank,Asset,Cash and Cash Equivalents
2,Bank of the Philippine Islands-004011022931,Cash in Bank,Asset,Cash and Cash Equivalents
3,Bank of the Philippine Islands-004011022958,Cash in Bank,Asset,Cash and Cash Equivalents
4,Bank of the Philippine Islands-004011022966,Cash in Bank,Asset,Cash and Cash Equivalents
...,...,...,...,...
261,Due to Unitholders,Due to Unitholders,Liability,Accrued Expenses and Other Liabilities
262,Final Tax Payable,Final Tax Payable,Liability,Accrued Expenses and Other Liabilities
263,Income Tax Payable,Income Tax Payable,Liability,Accrued Expenses and Other Liabilities
264,EWT,Withholding tax payable,Liability,Accrued Expenses and Other Liabilities


In [24]:
def merge_coa_data(
    unique_df: pd.DataFrame, 
    coa_mapping_df: pd.DataFrame,
    left_on_col: str = 'accountname',
    right_on_col: str = 'GL Account',
    mapping_cols: List[str] = ['TB Account Name', 'Account Type', 'FS Classification']
) -> pd.DataFrame:
    """
    Performs a left merge to attach CoA classification data to the unique account list.

    Args:
        unique_df: The target DataFrame (e.g., from create_unique_column_df) 
                   containing the unique keys.
        coa_mapping_df: The source DataFrame containing the mapping data.
        left_on_col: The key column in the unique_df (e.g., 'accountname').
        right_on_col: The key column in the coa_mapping_df (e.g., 'GL Account').
        mapping_cols: The columns to extract from coa_mapping_df and add to unique_df.

    Returns:
        The unique_df DataFrame augmented with the mapped columns.
    """
    # 1. Select only the necessary mapping columns plus the join key from the mapping table
    cols_to_select = [right_on_col] + mapping_cols
    mapping_subset = coa_mapping_df[cols_to_select]
    
    # 2. Perform a left merge
    # A left merge ensures all rows in unique_df (the unique accounts) are kept.
    merged_df = pd.merge(
        unique_df,
        mapping_subset,
        left_on=left_on_col,
        right_on=right_on_col,
        how='left'
    )
    
    # 3. Clean up the extra join key column that was merged (if keys are different)
    # The right_on_col is redundant since left_on_col already contains the key.
    if left_on_col != right_on_col and right_on_col in merged_df.columns:
        merged_df = merged_df.drop(columns=[right_on_col])
        
    return merged_df

In [25]:
coa_mapping

,GL Account,TB Account Name,Account Type,FS Classification
0,Bank of the Philippine Islands (BPI) - The For...,Cash in Bank,Asset,Cash and Cash Equivalents
1,Bank of the Philippine Islands-004011022923,Cash in Bank,Asset,Cash and Cash Equivalents
2,Bank of the Philippine Islands-004011022931,Cash in Bank,Asset,Cash and Cash Equivalents
3,Bank of the Philippine Islands-004011022958,Cash in Bank,Asset,Cash and Cash Equivalents
4,Bank of the Philippine Islands-004011022966,Cash in Bank,Asset,Cash and Cash Equivalents
...,...,...,...,...
261,Due to Unitholders,Due to Unitholders,Liability,Accrued Expenses and Other Liabilities
262,Final Tax Payable,Final Tax Payable,Liability,Accrued Expenses and Other Liabilities
263,Income Tax Payable,Income Tax Payable,Liability,Accrued Expenses and Other Liabilities
264,EWT,Withholding tax payable,Liability,Accrued Expenses and Other Liabilities


In [26]:
# 3. Merge the Classification Data
chart_of_accounts_final = merge_coa_data(unique_account_df, coa_mapping)

chart_of_accounts_final

,accountname,TB Account Name,Account Type,FS Classification
0,ANNUAL FEE - DIRECTOR,Annual Fee - Director,Expense,Director's Fee
1,Annual Fee - Director,Annual Fee - Director,Expense,Director's Fee
2,BIR Registration/License Fee,BIR Fees,Expense,Taxes and Licenses
3,Business Permit,Municipal Permit,Expense,Taxes and Licenses
4,Doc. Stamp Tax,Documentary Stamp Tax,Expense,Taxes and Licenses
...,...,...,...,...
504,Unrealized Gains/(Losses) - Foreign Currency,NaN,NaN,NaN
505,Vat Output - Others,VAT Payable,Expense,Accrued Expenses and Other Liabilities
506,Withdrawal-Interest,NaN,NaN,NaN
507,Withdrawal-Principal,NaN,NaN,NaN


In [27]:
# chart_of_accounts_final.to_excel('../data/processed/chart_of_accounts_11-12-2025.xlsx', index=False)

# Export Update COA MAPPING

In [28]:
def update_coa_mapping_and_save(
    chart_of_accounts_final: pd.DataFrame, 
    coa_mapping_df: pd.DataFrame,
    account_col: str = 'accountname',
    gl_account_col: str = 'GL Account',
    mapping_cols: List[str] = ['TB Account Name', 'Account Type', 'FS Classification'],
    save_path: str = '../data/references/COA Mapping folder',
    filename_base: str = 'Chart of Accounts Mapping'
) -> Tuple[pd.DataFrame, str]:
    """
    Identifies unclassified accounts, appends them to the CoA mapping table 
    with a placeholder value, and saves the updated mapping to a file.
    
    FIX: Now checks the save_path for the latest dated file and loads it 
    as the base for updates, ensuring manual classifications are preserved.

    Args:
        chart_of_accounts_final: The result of the merge_coa_data function 
                                 (unique accounts + classifications, potentially with NaNs).
        coa_mapping_df: The original CoA mapping DataFrame (used as a fallback if no saved file is found).
        account_col: The account name column in chart_of_accounts_final (default: 'accountname').
        gl_account_col: The key column in coa_mapping_df (default: 'GL Account').
        mapping_cols: The classification columns that might contain NaNs.
        save_path: The directory path to save the updated mapping file.
        filename_base: The base name for the output file (e.g., 'Chart of Accounts Mapping').

    Returns:
        A tuple containing: 
        1. The updated CoA mapping DataFrame.
        2. The full filepath of the saved document.
    """
    # 0. Check for and load the latest saved version first (to include manual updates)
    loaded_coa_mapping_df = coa_mapping_df.copy()
    
    # Create the directory if it doesn't exist (important for robust execution)
    os.makedirs(save_path, exist_ok=True)
    
    # Pattern to search for in filenames
    base_pattern = f"{filename_base} as of "

    # Helper function to extract date from filename
    def extract_date_from_filename(filename, base):
        try:
            # Example: "Chart of Accounts Mapping as of 11.14.2025.xlsx"
            date_str = filename.replace(base, "").replace(".xlsx", "")
            return datetime.strptime(date_str, '%m.%d.%Y')
        except ValueError:
            return datetime.min # Return minimum date for files that don't match the expected format

    files = os.listdir(save_path)
    candidate_files = [f for f in files if f.startswith(base_pattern) and f.endswith(".xlsx")]

    if candidate_files:
        latest_date = datetime.min
        latest_file = None
        
        for filename in candidate_files:
            current_date = extract_date_from_filename(filename, base_pattern)
            if current_date > latest_date:
                latest_date = current_date
                latest_file = filename
                
        if latest_file:
            latest_filepath = os.path.join(save_path, latest_file)
            try:
                # We assume the external file is the most up-to-date source of truth
                loaded_coa_mapping_df = pd.read_excel(latest_filepath)
                print(f"✅ Replaced input mapping with latest saved file: {latest_file}")
            except Exception as e:
                print(f"Error loading latest file {latest_filepath}. Proceeding with input coa_mapping_df. Error: {e}")


    # 1. Identify new records (where the first mapping column is NaN)
    # This assumes if the first mapping column is NaN, the rest are too (due to the merge).
    unclassified_accounts = chart_of_accounts_final[
        chart_of_accounts_final[mapping_cols[0]].isna()
    ].copy()

    if unclassified_accounts.empty:
        print("No new unclassified accounts found. Mapping table is up-to-date.")
        # Return the loaded file if it was loaded, or the original if not.
        return loaded_coa_mapping_df, "" 

    print(f"Found {len(unclassified_accounts)} new accounts to add to the mapping table.")

    # 2. Prepare new records for concatenation
    new_records = unclassified_accounts[[account_col]].copy()
    
    # Rename the account column to match the mapping table key ('GL Account')
    new_records.rename(columns={account_col: gl_account_col}, inplace=True)
    
    # Fill classification columns with placeholder value for immediate review
    # The original placeholder ' ' is used here, as per your previous version
    placeholder_value = ' ' 
    for col in mapping_cols:
        new_records[col] = placeholder_value
    
    # 3. Append new records to the loaded (or original) mapping table
    updated_coa_mapping_df = pd.concat([loaded_coa_mapping_df, new_records], ignore_index=True)
    
    # Remove duplicates from the GL Account key (in case an old unclassified was manually added, but reappears)
    updated_coa_mapping_df = updated_coa_mapping_df.drop_duplicates(subset=[gl_account_col], keep='last')
    
    # Optional: Sort the new mapping file by GL Account for better readability
    updated_coa_mapping_df = updated_coa_mapping_df.sort_values(by=gl_account_col).reset_index(drop=True)

    # 4. Save the updated mapping file
    date_exported = datetime.now().strftime('%m.%d.%Y')
    filename = f"{filename_base} as of {date_exported}.xlsx" 
    
    # Use os.path.join for cross-platform path construction
    full_filepath = os.path.join(save_path, filename)
    
    try:
        updated_coa_mapping_df.to_excel(full_filepath, index=False)
        print(f"Successfully saved updated mapping to: {full_filepath}")
    except Exception as e:
        print(f"Error saving file to {full_filepath}: {e}")
        full_filepath = f"Error saving file: {e}" # Indicate error in return path

    return updated_coa_mapping_df, full_filepath

In [29]:
# C. Load latest saved map OR use initial map, then update and save.
# We explicitly tell the function that the input key is 'accountname', but the output map must be 'GL Account'
master_mapping_df, saved_path = update_coa_mapping_and_save( # Changed function name here
    chart_of_accounts_final, 
    coa_mapping, 
    account_col='accountname', 
    gl_account_col='GL Account',
    save_path='../data/references/COA Mapping', # Using a local folder for the example
    filename_base='Chart of Accounts Mapping'
)


✅ Replaced input mapping with latest saved file: Chart of Accounts Mapping as of 10.10.2025.xlsx
Found 246 new accounts to add to the mapping table.
Successfully saved updated mapping to: ../data/references/COA Mapping\Chart of Accounts Mapping as of 11.21.2025.xlsx


In [30]:
coa_mapping

,GL Account,TB Account Name,Account Type,FS Classification
0,Bank of the Philippine Islands (BPI) - The For...,Cash in Bank,Asset,Cash and Cash Equivalents
1,Bank of the Philippine Islands-004011022923,Cash in Bank,Asset,Cash and Cash Equivalents
2,Bank of the Philippine Islands-004011022931,Cash in Bank,Asset,Cash and Cash Equivalents
3,Bank of the Philippine Islands-004011022958,Cash in Bank,Asset,Cash and Cash Equivalents
4,Bank of the Philippine Islands-004011022966,Cash in Bank,Asset,Cash and Cash Equivalents
...,...,...,...,...
261,Due to Unitholders,Due to Unitholders,Liability,Accrued Expenses and Other Liabilities
262,Final Tax Payable,Final Tax Payable,Liability,Accrued Expenses and Other Liabilities
263,Income Tax Payable,Income Tax Payable,Liability,Accrued Expenses and Other Liabilities
264,EWT,Withholding tax payable,Liability,Accrued Expenses and Other Liabilities


In [31]:
chart_of_accounts_final

,accountname,TB Account Name,Account Type,FS Classification
0,ANNUAL FEE - DIRECTOR,Annual Fee - Director,Expense,Director's Fee
1,Annual Fee - Director,Annual Fee - Director,Expense,Director's Fee
2,BIR Registration/License Fee,BIR Fees,Expense,Taxes and Licenses
3,Business Permit,Municipal Permit,Expense,Taxes and Licenses
4,Doc. Stamp Tax,Documentary Stamp Tax,Expense,Taxes and Licenses
...,...,...,...,...
504,Unrealized Gains/(Losses) - Foreign Currency,NaN,NaN,NaN
505,Vat Output - Others,VAT Payable,Expense,Accrued Expenses and Other Liabilities
506,Withdrawal-Interest,NaN,NaN,NaN
507,Withdrawal-Principal,NaN,NaN,NaN


In [32]:
# # Update the mapping and save the new file
# updated_coa_mapping, saved_path = update_coa_mapping_and_save(
#     chart_of_accounts_final, 
#     coa_mapping, 
#     save_path='../data/references/COA Mapping', # Using a local folder for the example
#     filename_base='Chart of Accounts Mapping'
# )

In [33]:
# Load reference data
reference_data = load_reference_data()

# Reference data
coa_mapping = reference_data['coa_mapping']

coa_mapping

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "C:\Users\raiden.guillergan\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4c2' in position 33: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\UserProfile\Documents\@ VFC\pemi-automation\trial-balance\.venv\Lib\site-packages\traitlets\config\application.p

,GL Account,TB Account Name,Account Type,FS Classification
0,ANNUAL FEE - DIRECTOR,Annual Fee - Director,Expense,Director's Fee
1,Annual Fee - Director,Annual Fee - Director,Expense,Director's Fee
2,BIR Registration/License Fee,BIR Fees,Expense,Taxes and Licenses
3,Bank Charges,Bank Charges,Expense,Others
4,Business Permit,Municipal Permit,Expense,Taxes and Licenses
...,...,...,...,...
507,Unrealized Gains/(Losses) - Foreign Currency,,,
508,Vat Output - Others,VAT Payable,Expense,Accrued Expenses and Other Liabilities
509,Withdrawal-Interest,,,
510,Withdrawal-Principal,,,


## 9. Match GL Accounts with COA Mapping

In [34]:
# # Get GL Accounts from pivot table (index)
# pivot_gl_accounts = set(trial_balance_pivot_table.index)

# # Get GL Accounts from COA Mapping
# coa_gl_accounts = set(coa_mapping['GL Account'])

# # Find accounts in pivot table that are NOT in COA Mapping
# missing_in_coa = pivot_gl_accounts - coa_gl_accounts

# # Find accounts in COA Mapping that are NOT in pivot table
# missing_in_pivot = coa_gl_accounts - pivot_gl_accounts

# print("="*60)
# print("GL ACCOUNT MATCHING ANALYSIS")
# print("="*60)
# print(f"\n📊 Total GL Accounts in Pivot Table: {len(pivot_gl_accounts)}")
# print(f"📊 Total GL Accounts in COA Mapping: {len(coa_gl_accounts)}")
# print(f"\n✓ Matching Accounts: {len(pivot_gl_accounts & coa_gl_accounts)}")
# print(f"⚠️  Accounts in Pivot but NOT in COA Mapping: {len(missing_in_coa)}")
# print(f"ℹ️  Accounts in COA Mapping but NOT in Pivot: {len(missing_in_pivot)}")

# # Display missing accounts
# if missing_in_coa:
#     print("\n" + "="*60)
#     print("⚠️  NEW ACCOUNTS FOUND (Need to be added to COA Mapping):")
#     print("="*60)
#     for i, account in enumerate(sorted(missing_in_coa), 1):
#         print(f"{i:3}. {account}")
# else:
#     print("\n✓ All accounts in pivot table exist in COA Mapping!")

# # Create indicator DataFrame for new accounts
# if missing_in_coa:
#     new_accounts_df = pd.DataFrame({
#         'GL Account': sorted(missing_in_coa),
#         'Status': 'NEW - Not in COA Mapping',
#         'TB Account Name': '',
#         'Account Type': '',
#         'FS Classification': ''
#     })
    
#     print(f"\n📝 Created DataFrame with {len(new_accounts_df)} new account(s) to be added")
#     print("    Variable: new_accounts_df")
# else:
#     new_accounts_df = None
#     print("\n✓ No new accounts to add")

In [35]:
# # Display new accounts DataFrame
# if new_accounts_df is not None:
#     print(f"📋 New Accounts to Add to COA Mapping ({len(new_accounts_df)} accounts):\n")
#     display(new_accounts_df)
# else:
#     print("✓ No new accounts found")

In [36]:
# # Create updated COA Mapping with new accounts inserted
# if new_accounts_df is not None:
#     # Combine original COA mapping with new accounts
#     updated_coa_mapping = pd.concat([coa_mapping, new_accounts_df], ignore_index=True)
    
#     # Sort by GL Account for better organization
#     updated_coa_mapping = updated_coa_mapping.sort_values('GL Account').reset_index(drop=True)
    
#     print("✓ Updated COA Mapping created with new accounts")
#     print(f"\n📊 Original COA Mapping: {len(coa_mapping)} accounts")
#     print(f"📊 New Accounts Added: {len(new_accounts_df)} accounts")
#     print(f"📊 Updated COA Mapping: {len(updated_coa_mapping)} accounts")
#     print(f"\n💾 Variable: updated_coa_mapping")
    
#     # Create indicator column to show which accounts are new
#     updated_coa_mapping['Is_New_Account'] = updated_coa_mapping['GL Account'].isin(missing_in_coa)
    
#     print(f"\n✓ Added 'Is_New_Account' indicator column")
#     print(f"   - True: Account is newly found (not in original COA Mapping)")
#     print(f"   - False: Account existed in original COA Mapping")
# else:
#     updated_coa_mapping = coa_mapping.copy()
#     updated_coa_mapping['Is_New_Account'] = False
#     print("✓ No new accounts to add - using original COA Mapping")

In [37]:
# # Display updated COA Mapping - showing only new accounts
# print("📋 Updated COA Mapping - New Accounts Only:\n")
# display(updated_coa_mapping[updated_coa_mapping['Is_New_Account'] == True])

In [38]:
# # Export updated COA Mapping if new accounts were added
# if new_accounts_df is not None and len(new_accounts_df) > 0:
#     # Define export path
#     export_folder = Path('../data/references/COA Mapping')
#     export_folder.mkdir(parents=True, exist_ok=True)
    
#     # Create filename with current date (MM.DD.YYYY format)
#     current_date = datetime.now().strftime('%m.%d.%Y')
#     export_filename = f'Chart of Accounts Mapping as of {current_date}.xlsx'
#     export_path = export_folder / export_filename
    
#     # Export to Excel
#     updated_coa_mapping.to_excel(export_path, index=False, engine='openpyxl')
    
#     print("="*60)
#     print("📤 EXPORT SUCCESSFUL")
#     print("="*60)
#     print(f"✓ File exported to: {export_path}")
#     print(f"✓ Filename: {export_filename}")
#     print(f"✓ Total records: {len(updated_coa_mapping)}")
#     print(f"✓ New accounts added: {len(new_accounts_df)}")
#     print(f"✓ Export timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
#     print("\n💡 Note: The 'Is_New_Account' column indicates which accounts are newly added (True)")
# else:
#     print("ℹ️  No new accounts to export - COA Mapping unchanged")

In [39]:
# chart_of_accounts

In [40]:
# coa_mapping

In [41]:
# chart_of_accounts_final

In [42]:
def capitalize_portfolio_mapping(portfolio_df):
    """
    Capitalize all values in the 'level1accountname' column of the portfolio mapping.
    
    Parameters:
        portfolio_df (DataFrame): The portfolio mapping DataFrame
    
    Returns:
        DataFrame: Portfolio mapping with capitalized 'level1accountname' values, or None if input is None
    """
    # Handle None case
    if portfolio_df is None:
        print("⚠️  WARNING: portfolio_mapping is None - no data to capitalize")
        return None
    
    # Create a copy to avoid modifying original
    df_copy = portfolio_df.copy()
    
    # Find the correct column name (it has a tab character)
    level1_col = [col for col in df_copy.columns if 'level1accountname' in col.lower()]
    
    if level1_col:
        col_name = level1_col[0]
        df_copy[col_name] = df_copy[col_name].str.upper()
        print(f"✓ Portfolio mapping '{col_name}' column capitalized")
        print(f"  Updated {len(df_copy)} row(s)")
        print(f"  Unique values: {df_copy[col_name].nunique()}")
    else:
        print("⚠️  WARNING: 'level1accountname' column not found in portfolio_mapping")
    
    return df_copy

# Apply the function to portfolio_mapping
portfolio_mapping = capitalize_portfolio_mapping(portfolio_mapping)

# Display updated portfolio mapping
if portfolio_mapping is not None and len([col for col in portfolio_mapping.columns if 'level1accountname' in col.lower()]) > 0:
    level1_col = [col for col in portfolio_mapping.columns if 'level1accountname' in col.lower()][0]
    print(f"\n📋 Updated portfolio_mapping unique values:")
    print(portfolio_mapping[level1_col].unique().tolist())
else:
    print("\nℹ️  No portfolio mapping data available to display")

✓ Portfolio mapping 'level1accountname	' column capitalized
  Updated 16 row(s)
  Unique values: 16

📋 Updated portfolio_mapping unique values:
['PHILEQUITY ALPHA ONE FUND, INC.', 'PHILEQUITY DIVIDEND YIELD FUND\t', 'PHILEQUITY DOLLAR INCOME FUND\t', 'PHILEQUITY FUND, INC.\t', 'PHILEQUITY MSCI PHILIPPINES INDEX FUND, INC.\t', 'PHILEQUITY PESO BOND FUND\t', 'PHILEQUITY PSE INDEX FUND\t', 'GOVERNMENT SERVICE INSURANCE SYSTEM-500032', 'PHILEQUITY MANAGEMENT, INC.\t', 'PHILEQUITY ALPHA ONE FUND, INC. S\t', 'BANGKO SENTRAL NG PILIPINAS PROVIDENT FUND (BSPPF)-607896\t', 'DE LA SALLE - COLLEGE OF SAINT BENILDE, INC. (DLSU-CSB)-607894\t', 'DE LA SALLE UNIVERSITY, INC. (DLSU)-607892\t', 'GOVERNMENT SERVICE INSURANCE SYSTEM IMA (GSIS)-607898\t', 'PHILEQUITY MGT INC. IMA#008 - 85399-607902\t', 'SOCIAL SECURITY SYSTEM IMA (SSS)-607900\t']


In [43]:
portfolio_mapping

,level1accountname\t,Fund_Code
0,"PHILEQUITY ALPHA ONE FUND, INC.",PAOF
1,PHILEQUITY DIVIDEND YIELD FUND\t,PDYFI
2,PHILEQUITY DOLLAR INCOME FUND\t,PDIF
3,"PHILEQUITY FUND, INC.\t",PEFI
4,"PHILEQUITY MSCI PHILIPPINES INDEX FUND, INC.\t",PMPI
5,PHILEQUITY PESO BOND FUND\t,PPBF
6,PHILEQUITY PSE INDEX FUND\t,PPSE
7,GOVERNMENT SERVICE INSURANCE SYSTEM-500032,OLD-GSIS
8,"PHILEQUITY MANAGEMENT, INC.\t",PEMI
9,"PHILEQUITY ALPHA ONE FUND, INC. S\t",PAOFS


In [44]:
trial_balance_pivot_table

,BANGKO SENTRAL NG PILIPINAS PROVIDENT FUND (BSPPF)-607896,"DE LA SALLE - COLLEGE OF SAINT BENILDE, INC. (DLSU-CSB)-607894","DE LA SALLE UNIVERSITY, INC. (DLSU)-607892",GOVERNMENT SERVICE INSURANCE SYSTEM-500032,GOVERNMENT SERVICE INSURANCE SYSTEM IMA (GSIS)-607898,PHILEQUITY DIVIDEND YIELD FUND,PHILEQUITY DOLLAR INCOME FUND,"PHILEQUITY FUND, INC.","PHILEQUITY MANAGEMENT, INC.",PHILEQUITY PESO BOND FUND,PHILEQUITY PSE INDEX FUND,"PHILEQUITY ALPHA ONE FUND, INC.","PHILEQUITY ALPHA ONE FUND, INC. S","PHILEQUITY MSCI PHILIPPINES INDEX FUND, INC.",PHILEQUITY MGT INC. IMA#008 - 85399-607902,SOCIAL SECURITY SYSTEM IMA (SSS)-607900
GL Account,,,,,,,,,,,,,,,,
ANNUAL FEE - DIRECTOR,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Annual Fee - Director,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BIR Registration/License Fee,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-3000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Business Permit,0.00,0.00,0.00,0.00,0.00,0.00,-25.50,-11283543.05,0.00,-0.13,0.00,0.00,0.00,0.00,0.00,0.00
Doc. Stamp Tax,0.00,0.00,0.00,0.00,0.00,-3373495.28,-0.00,-1358391.92,0.00,-0.12,-0.03,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Union Bank of the Philippines, Inc.-6101011305",0.00,0.00,0.00,0.00,0.00,0.00,1974222.28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Unrealized Gain / Loss,-55505407.93,14759960.46,-43211680.26,0.00,58149555.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,92292705543.68,77587036.34
Vat Output - Others,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-6253161246.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [45]:
def rename_pivot_columns_with_fund_code_v2(pivot_df, portfolio_df):
    """
    Rename pivot table columns based on portfolio mapping.
    Maps level1accountname to Fund_Code from portfolio_mapping using 
    case-insensitive and whitespace-agnostic matching (UPPERCASE + Strip).
    
    Parameters:
        pivot_df (DataFrame): The pivot table with fund names as columns
        portfolio_df (DataFrame): The portfolio mapping DataFrame
    
    Returns:
        DataFrame: Pivot table with renamed columns (Fund Codes)
    """
    if portfolio_df is None:
        print("⚠️  WARNING: portfolio_mapping is None - cannot rename columns")
        return pivot_df
    
    # Create a copy to avoid modifying original
    df_copy = pivot_df.copy()
    
    # --- 1. Find the necessary columns (same as original) ---
    level1_col = [col for col in portfolio_df.columns if 'level1accountname' in col.lower()]
    fund_code_col = [col for col in portfolio_df.columns if 'fund_code' in col.lower() or 'fund code' in col.lower()]
    
    if not level1_col or not fund_code_col:
        print(f"⚠️  WARNING: Required column not found. Level1 found: {bool(level1_col)}, Fund_Code found: {bool(fund_code_col)}")
        return pivot_df
    
    level1_col_name = level1_col[0]
    fund_code_col_name = fund_code_col[0]
    
    # --- 2. Create MAPPING DICTIONARY (Key: UPPER/Strip Fund Name, Value: Fund Code) ---
    # The key is the standardized name we expect from pivot_df columns
    mapping_dict = dict(zip(
        portfolio_df[level1_col_name].str.strip().str.upper(),
        portfolio_df[fund_code_col_name]
    ))
    
    # --- 3. Rename Columns: Apply Standardization for Lookup ---
    
    # The core change is here: we standardize the column name (col) before looking it up.
    def get_new_col_name(old_col_name):
        # Standardize the pivot table column name to match the dictionary key
        standardized_name = str(old_col_name).strip().upper()
        
        # Look up the new name. If not found, return the original column name.
        return mapping_dict.get(standardized_name, old_col_name)

    new_columns = [get_new_col_name(col) for col in df_copy.columns]
    
    df_copy.columns = new_columns
    
    # --- 4. Logging and Return (same as original, adjusted for v2) ---
    
    # Optional: You can still print the mapping dictionary for verification
    print(f"📋 Column Mapping Dictionary (Keys are UPPER/Strip):")
    for old_name, new_name in list(mapping_dict.items())[:5]: # Print first 5 for brevity
        print(f"  {old_name} → {new_name}")
    if len(mapping_dict) > 5:
        print("  ...")
    
    renamed_count = sum(1 for old, new in zip(pivot_df.columns, df_copy.columns) if old != new)

    print(f"\n✓ Pivot table columns renamed")
    print(f"  Original columns (first 5): {list(pivot_df.columns)[:5]}")
    print(f"  New columns (first 5): {list(df_copy.columns)[:5]}")
    print(f"  Renamed {renamed_count} column(s)")
    
    return df_copy

In [46]:
# Apply the function to rename columns
trial_balance_pivot_table = rename_pivot_columns_with_fund_code_v2(trial_balance_pivot_table, portfolio_mapping)

# Display updated pivot table
print(f"\n📊 Updated Pivot Table Columns:")
print(list(trial_balance_pivot_table.columns)) 



📋 Column Mapping Dictionary (Keys are UPPER/Strip):
  PHILEQUITY ALPHA ONE FUND, INC. → PAOF
  PHILEQUITY DIVIDEND YIELD FUND → PDYFI
  PHILEQUITY DOLLAR INCOME FUND → PDIF
  PHILEQUITY FUND, INC. → PEFI
  PHILEQUITY MSCI PHILIPPINES INDEX FUND, INC. → PMPI
  ...

✓ Pivot table columns renamed
  Original columns (first 5): ['BANGKO SENTRAL NG PILIPINAS PROVIDENT FUND (BSPPF)-607896', 'DE LA SALLE - COLLEGE OF SAINT BENILDE, INC. (DLSU-CSB)-607894', 'DE LA SALLE UNIVERSITY, INC. (DLSU)-607892', 'GOVERNMENT SERVICE INSURANCE SYSTEM-500032', 'GOVERNMENT SERVICE INSURANCE SYSTEM IMA (GSIS)-607898']
  New columns (first 5): ['BSP', 'DLS-CSB', 'DLSU', 'OLD-GSIS', 'GSIS']
  Renamed 16 column(s)

📊 Updated Pivot Table Columns:
['BSP', 'DLS-CSB', 'DLSU', 'OLD-GSIS', 'GSIS', 'PDYFI', 'PDIF', 'PEFI', 'PEMI', 'PPBF', 'PPSE', 'PAOF', 'PAOFS', 'PMPI', 'IMA 008', 'SSS']


In [47]:
portfolio_mapping

,level1accountname\t,Fund_Code
0,"PHILEQUITY ALPHA ONE FUND, INC.",PAOF
1,PHILEQUITY DIVIDEND YIELD FUND\t,PDYFI
2,PHILEQUITY DOLLAR INCOME FUND\t,PDIF
3,"PHILEQUITY FUND, INC.\t",PEFI
4,"PHILEQUITY MSCI PHILIPPINES INDEX FUND, INC.\t",PMPI
5,PHILEQUITY PESO BOND FUND\t,PPBF
6,PHILEQUITY PSE INDEX FUND\t,PPSE
7,GOVERNMENT SERVICE INSURANCE SYSTEM-500032,OLD-GSIS
8,"PHILEQUITY MANAGEMENT, INC.\t",PEMI
9,"PHILEQUITY ALPHA ONE FUND, INC. S\t",PAOFS


In [48]:
list(trial_balance_pivot_table.columns)

['BSP',
 'DLS-CSB',
 'DLSU',
 'OLD-GSIS',
 'GSIS',
 'PDYFI',
 'PDIF',
 'PEFI',
 'PEMI',
 'PPBF',
 'PPSE',
 'PAOF',
 'PAOFS',
 'PMPI',
 'IMA 008',
 'SSS']

In [49]:
# Display the renamed pivot table
trial_balance_pivot_table

,BSP,DLS-CSB,DLSU,OLD-GSIS,GSIS,PDYFI,PDIF,PEFI,PEMI,PPBF,PPSE,PAOF,PAOFS,PMPI,IMA 008,SSS
GL Account,,,,,,,,,,,,,,,,
ANNUAL FEE - DIRECTOR,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Annual Fee - Director,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BIR Registration/License Fee,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-3000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Business Permit,0.00,0.00,0.00,0.00,0.00,0.00,-25.50,-11283543.05,0.00,-0.13,0.00,0.00,0.00,0.00,0.00,0.00
Doc. Stamp Tax,0.00,0.00,0.00,0.00,0.00,-3373495.28,-0.00,-1358391.92,0.00,-0.12,-0.03,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Union Bank of the Philippines, Inc.-6101011305",0.00,0.00,0.00,0.00,0.00,0.00,1974222.28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Unrealized Gain / Loss,-55505407.93,14759960.46,-43211680.26,0.00,58149555.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,92292705543.68,77587036.34
Vat Output - Others,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-6253161246.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [50]:
def drop_zero_value_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filters and removes rows from a DataFrame where the sum of all numerical 
    columns is zero. This efficiently removes 'zero-value records' without
    creating or dropping temporary columns.

    Parameters:
        df (pd.DataFrame): The input DataFrame (e.g., a pivot table).

    Returns:
        pd.DataFrame: The DataFrame with rows containing only zero numerical
                      values removed.
    """
    # 1. Select only numerical columns (robust handling for mixed types)
    # This prevents errors if non-numeric columns (like account descriptions) exist.
    numeric_df = df.select_dtypes(include=np.number)

    if numeric_df.empty:
        print("⚠️  WARNING: No numerical columns found to sum. Returning original DataFrame.")
        return df

    # 2. Calculate the row sum (axis=1) of only the numerical columns.
    # This Series acts as the 'helper_col' logic but without adding it to the DF.
    row_sum = numeric_df.sum(axis=1)
    
    # 3. Create a boolean mask: keep rows where the absolute sum is NOT zero.
    # We use abs() to handle potential tiny floating point errors, though often 
    # unnecessary for standard pivot tables.
    mask = (row_sum.abs() != 0)
    
    # 4. Apply the mask to the original DataFrame and return the filtered copy.
    df_cleaned = df[mask].copy()
    
    print(f"\n🗑️  Row Filtering Complete")
    print(f"  Original Rows: {len(df)}")
    print(f"  Rows Retained: {len(df_cleaned)}")
    print(f"  Rows Dropped (Zero Value): {len(df) - len(df_cleaned)}")
    
    return df_cleaned

In [51]:
# Apply the function to drop zero-value rows
trial_balance_pivot_table = drop_zero_value_rows(trial_balance_pivot_table)


🗑️  Row Filtering Complete
  Original Rows: 313
  Rows Retained: 261
  Rows Dropped (Zero Value): 52


In [52]:
trial_balance_pivot_table

,BSP,DLS-CSB,DLSU,OLD-GSIS,GSIS,PDYFI,PDIF,PEFI,PEMI,PPBF,PPSE,PAOF,PAOFS,PMPI,IMA 008,SSS
GL Account,,,,,,,,,,,,,,,,
ANNUAL FEE - DIRECTOR,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BIR Registration/License Fee,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-3000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Business Permit,0.00,0.00,0.00,0.00,0.00,0.00,-25.50,-11283543.05,0.00,-0.13,0.00,0.00,0.00,0.00,0.00,0.00
Doc. Stamp Tax,0.00,0.00,0.00,0.00,0.00,-3373495.28,-0.00,-1358391.92,0.00,-0.12,-0.03,0.00,0.00,0.00,0.00,0.00
Legal & Professsional Fees,0.00,0.00,0.00,0.00,0.00,-0.03,-5396.54,6231846.44,0.00,0.00,2426861.40,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Union Bank of the Philippines, Inc.-6101011305",0.00,0.00,0.00,0.00,0.00,0.00,1974222.28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Unrealized Gain / Loss,-55505407.93,14759960.46,-43211680.26,0.00,58149555.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,92292705543.68,77587036.34
Vat Output - Others,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-6253161246.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [53]:
def merge_coa_details(df: pd.DataFrame, coa_df: pd.DataFrame, merge_cols: List[str]) -> pd.DataFrame:
    """
    Merges COA details ('TB Account Name', 'Account Type') into the pivot table
    based on 'GL Account' and places the new columns as the initial columns.
    
    The merge is performed using a standardized (stripped, upper) temporary key 
    to ensure case- and whitespace-agnostic matching.
    
    FIX: Now includes reset_index() for the pivot table, as the GL Account 
    is typically in the Index after pivoting.

    Parameters:
        df (pd.DataFrame): The target pivot table (e.g., trial_balance_pivot_table).
        coa_df (pd.DataFrame): The source COA mapping DataFrame.
        merge_cols (List[str]): List of columns to insert from coa_df (e.g., ['TB Account Name', 'Account Type']).

    Returns:
        pd.DataFrame: DataFrame with COA details merged and reordered.
    """
    print("\n🔗 Starting COA Details Merge...")
    
    # --- PIVOT TABLE FIX: Move Index (GL Account) to a Column ---
    # We assume GL Account is in the index due to the KeyError and the column names
    if df.index.name is not None:
        df = df.reset_index()
    elif df.index.names != [None] * len(df.index.names):
        # Handle MultiIndex case
        df = df.reset_index()
    else:
        # If the index is unnamed, we must guess it's the first column in the coa_df
        # We rely on the name standardization below to correctly pick up the column name
        pass

    # --- ULTIMATE FIX: Standardize all column names first (Case + Strip) ---
    # Convert all column names to stripped UPPERCASE. This makes column access robust.
    df.columns = [str(col).strip().upper() for col in df.columns]
    coa_df.columns = [str(col).strip().upper() for col in coa_df.columns]
    
    # --- CONFIRMATION OF STANDARDIZATION (Internal Check) ---
    print(f"  DF Columns after standardization: {df.columns.tolist()[:3]}...")
    print(f"  COA Columns after standardization: {coa_df.columns.tolist()[:3]}...")
    # --------------------------------------------------------

    # Define the standardized column name for GL Account
    GL_ACCOUNT_COL = 'GL ACCOUNT'
    
    # Standardize the merge_cols so they can be looked up in the coa_df

    # --- 1. Robust Key Creation ---
    coa_df = coa_df.copy() # Work on copy of coa_df to add temp column
    
    # Now that the column names are standardized AND index is reset, we access them using GL_ACCOUNT_COL
    # We still strip/upper the VALUE within the column for the robust merge KEY
    df['__merge_key'] = df[GL_ACCOUNT_COL].astype(str).str.strip().str.upper()
    coa_df['__merge_key'] = coa_df[GL_ACCOUNT_COL].astype(str).str.strip().str.upper()
    
    # --- 2. Prepare COA subset for merge ---
    
    # Create a list of columns to retrieve from COA, using their uppercase names
    # to access the DataFrame, but using the original name (or close match) 
    # to maintain the desired output header names.
    
    coa_upper_cols = [str(col).strip().upper() for col in merge_cols]
    
    # The actual columns we will merge: the merge key + the COA columns
    coa_final_cols = ['__merge_key'] + coa_upper_cols
    coa_subset = coa_df[coa_final_cols].drop_duplicates(subset=['__merge_key']).copy()
    
    # Rename the columns in the subset back to the desired output names for the merge
    # We rename the uppercase columns back to the mixed-case names (e.g. 'TB ACCOUNT NAME' -> 'TB Account Name')
    coa_rename_dict = dict(zip(coa_upper_cols, merge_cols))
    coa_subset = coa_subset.rename(columns=coa_rename_dict)
    
    # --- 3. Perform Left Merge ---
    merged_df = pd.merge(
        df, coa_subset,
        on='__merge_key',
        how='left',
        validate='many_to_one'
    )
    
    # Drop the temporary merge key column
    merged_df = merged_df.drop(columns=['__merge_key'])
    
    # --- 4. Column Reordering ---
    
    # We need to find the standardized GL Account column and put it after the new columns.
    
    # The list of columns we want to be placed first
    initial_cols = merge_cols + [col for col in merged_df.columns if col == GL_ACCOUNT_COL]

    # Get the rest of the columns
    remaining_cols = [col for col in merged_df.columns if col not in initial_cols and col not in merge_cols]
            
    # Define the final order: new columns, GL Account, then the rest
    final_cols = initial_cols + remaining_cols
    
    df_final = merged_df[final_cols].copy()
    
    # # Sort the result by the primary descriptive column (as requested) ---
    # sort_column = merge_cols[0]
    # df_final = df_final.sort_values(by=sort_column).reset_index(drop=True)
    
    print(f"✓ Merge complete. Columns inserted: {merge_cols}")
    print(f"  New initial columns: {df_final.columns[:len(merge_cols) + 1].tolist()}")
    
    return df_final

In [54]:
# 2. Apply the merge and reorder function
trial_balance_pivot_table = merge_coa_details(
    trial_balance_pivot_table, 
    coa_mapping, 
    ['TB Account Name', 'Account Type']
)

trial_balance_pivot_table


🔗 Starting COA Details Merge...
  DF Columns after standardization: ['GL ACCOUNT', 'BSP', 'DLS-CSB']...
  COA Columns after standardization: ['GL ACCOUNT', 'TB ACCOUNT NAME', 'ACCOUNT TYPE']...
✓ Merge complete. Columns inserted: ['TB Account Name', 'Account Type']
  New initial columns: ['TB Account Name', 'Account Type', 'GL ACCOUNT']


,TB Account Name,Account Type,GL ACCOUNT,BSP,DLS-CSB,DLSU,OLD-GSIS,GSIS,PDYFI,PDIF,PEFI,PEMI,PPBF,PPSE,PAOF,PAOFS,PMPI,IMA 008,SSS
0,Annual Fee - Director,Expense,ANNUAL FEE - DIRECTOR,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,BIR Fees,Expense,BIR Registration/License Fee,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-3000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Municipal Permit,Expense,Business Permit,0.00,0.00,0.00,0.00,0.00,0.00,-25.50,-11283543.05,0.00,-0.13,0.00,0.00,0.00,0.00,0.00,0.00
3,Documentary Stamp Tax,Expense,Doc. Stamp Tax,0.00,0.00,0.00,0.00,0.00,-3373495.28,-0.00,-1358391.92,0.00,-0.12,-0.03,0.00,0.00,0.00,0.00,0.00
4,Professional Fees,Expense,Legal & Professsional Fees,0.00,0.00,0.00,0.00,0.00,-0.03,-5396.54,6231846.44,0.00,0.00,2426861.40,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Cash in Bank,Asset,"Union Bank of the Philippines, Inc.-6101011305",0.00,0.00,0.00,0.00,0.00,0.00,1974222.28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
257,,,Unrealized Gain / Loss,-55505407.93,14759960.46,-43211680.26,0.00,58149555.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,92292705543.68,77587036.34
258,VAT Payable,Expense,Vat Output - Others,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-6253161246.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00
259,,,Withdrawal-Principal,1530000000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42877500000.00,0.00


In [55]:
trial_balance_pivot_table.columns

Index(['TB Account Name', 'Account Type', 'GL ACCOUNT', 'BSP', 'DLS-CSB',
       'DLSU', 'OLD-GSIS', 'GSIS', 'PDYFI', 'PDIF', 'PEFI', 'PEMI', 'PPBF',
       'PPSE', 'PAOF', 'PAOFS', 'PMPI', 'IMA 008', 'SSS'],
      dtype='object')

# Creation of TB FS

In [56]:
def create_deduplicated_mapping_subset(
    df: pd.DataFrame, 
    subset_cols: List[str], 
    deduplicate_on_col: str
) -> pd.DataFrame:
    """
    Creates a new DataFrame containing a subset of columns, deduplicated based 
    on a specified key column.

    This is useful for extracting unique high-level classifications 
    (like TB Account Name and Account Type) from a detailed transaction/pivot table 
    or a COA mapping table.

    Args:
        df (pd.DataFrame): The source DataFrame (e.g., final merged pivot table).
        subset_cols (List[str]): The list of columns to include in the output DF.
        deduplicate_on_col (str): The column used to drop duplicates.

    Returns:
        pd.DataFrame: A DataFrame with the unique combinations of the subset columns, 
                      sorted by the deduplicate key.
    """
    
    # 1. Validate required columns exist
    missing_cols = [col for col in subset_cols if col not in df.columns]
    if missing_cols:
        print(f"Error: Missing required columns for subset: {missing_cols}")
        return pd.DataFrame()
        
    # 2. Select the subset and create a copy
    subset_df = df[subset_cols].copy()

    # 3. Deduplicate based on the primary key, keeping the first occurrence
    # If there are conflicts (same deduplicate_on_col with different values in other subset_cols),
    # this will keep the first instance encountered.
    tb_fs_df = subset_df.drop_duplicates(
        subset=[deduplicate_on_col], 
        keep='first'
    ).reset_index(drop=True)
    
    # 4. Sort the result for clean presentation
    tb_fs_df = tb_fs_df.sort_values(by=deduplicate_on_col).reset_index(drop=True)
    
    return tb_fs_df

In [57]:
trial_balance_consolidated

,bookname,level1accountname,level2accountname,accountname,Portfolio,Portcode,Ext. Portfolio Code,Ext. Portcode 2,openingdebit,openingcredit,perioddebit,periodcredit,closingdebit,closingcredit,netamt,GL Code,cust_accountid,Account ID,accounttype,accountsubtype,descr,Entity,Date
0,MAIN ACCOUNT,Bangko Sentral Ng Pilipinas Provident Fund (bs...,Asset,A/R - Others PHP,Bangko Sentral Ng Pilipinas Provident Fund (bs...,607896,NaN,NaN,1008.49,1008.49,0.00,0.00,1008.49,1008.49,0.00,NaN,16204,16204,B/S,A,INVESTMENT,NaN,2025-09-01
1,MAIN ACCOUNT,Bangko Sentral Ng Pilipinas Provident Fund (bs...,Asset,ACCRUED INTEREST RECEIVABLE,Bangko Sentral Ng Pilipinas Provident Fund (bs...,607896,NaN,NaN,106763.37,106763.37,0.00,0.00,106763.37,106763.37,0.00,NaN,16149,16149,B/S,A,INVESTMENT,NaN,2025-09-01
2,MAIN ACCOUNT,Bangko Sentral Ng Pilipinas Provident Fund (bs...,Asset,AIR - MMP - PHP,Bangko Sentral Ng Pilipinas Provident Fund (bs...,607896,NaN,NaN,209773.51,209773.51,3228.33,0.00,213001.84,209773.51,3228.33,NaN,16369,16369,B/S,A,INVESTMENT,NaN,2025-09-01
3,MAIN ACCOUNT,Bangko Sentral Ng Pilipinas Provident Fund (bs...,Asset,DUE FROM BROKERS,Bangko Sentral Ng Pilipinas Provident Fund (bs...,607896,NaN,NaN,54701864.23,54701864.23,3629042.67,0.00,58330906.90,54701864.23,3629042.67,NaN,16367,16367,B/S,A,INVESTMENT,NaN,2025-09-01
4,MAIN ACCOUNT,Bangko Sentral Ng Pilipinas Provident Fund (bs...,Asset,Dividend Receivable,Bangko Sentral Ng Pilipinas Provident Fund (bs...,607896,NaN,NaN,15387963.51,12854473.67,0.00,0.00,15387963.51,12854473.67,2533489.84,NaN,16320,16320,B/S,A,INVESTMENT,NaN,2025-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40157,MAIN ACCOUNT,Social Security System Ima (sss)-607900,Income,Int. Income - CASA Bank Deposits-Tax-Pd (net)-PhP,Social Security System Ima (sss)-607900,607900,NaN,NaN,0.00,9319.86,0.00,0.00,0.00,9319.86,-9319.86,NaN,16895,16895,P/L,I,INCOME,NaN,2025-09-30
40158,MAIN ACCOUNT,Social Security System Ima (sss)-607900,Income,Trading Gain / Loss,Social Security System Ima (sss)-607900,607900,NaN,NaN,1398560.00,11153168.26,0.00,0.00,1398560.00,11153168.26,-9754608.26,NaN,17019,17019,P/L,E,EXPENSE,NaN,2025-09-30
40159,MAIN ACCOUNT,Social Security System Ima (sss)-607900,Income,Unrealized Gain / Loss,Social Security System Ima (sss)-607900,607900,NaN,NaN,74807865735.52,74773552443.83,335201253.77,325544964.47,75143066989.29,75099097408.30,43969580.99,NaN,17017,17017,P/L,I,INCOME,NaN,2025-09-30
40160,MAIN ACCOUNT,Social Security System Ima (sss)-607900,Liability,A/P - Other,Social Security System Ima (sss)-607900,607900,NaN,NaN,9314935.00,9314935.00,0.00,0.00,9314935.00,9314935.00,0.00,NaN,16774,16774,B/S,L,CURRENT LIABILITIES,NaN,2025-09-30


In [58]:
# --- STAGE 3: CREATE DEDUPLICATED MAPPING SUBSET (Your request) ---
tb_fs_df = create_deduplicated_mapping_subset(
    df=trial_balance_pivot_table,
    subset_cols=['TB Account Name', 'Account Type'],
    deduplicate_on_col='TB Account Name'
)

tb_fs_df 


,TB Account Name,Account Type
0,,
1,A/P - Others,Liability
2,Accounts Receivable,Asset
3,Accrued Exp. Payable,Expense
4,Accrued interest Receivable,Asset
5,Additional Paid-In Capital,Equity
6,Allowance for Prob. Losses,Asset
7,Annual Fee - Director,Expense
8,BIR Fees,Expense
9,Bank Charges,Expense


In [59]:
coa_mapping

,GL ACCOUNT,TB ACCOUNT NAME,ACCOUNT TYPE,FS CLASSIFICATION
0,ANNUAL FEE - DIRECTOR,Annual Fee - Director,Expense,Director's Fee
1,Annual Fee - Director,Annual Fee - Director,Expense,Director's Fee
2,BIR Registration/License Fee,BIR Fees,Expense,Taxes and Licenses
3,Bank Charges,Bank Charges,Expense,Others
4,Business Permit,Municipal Permit,Expense,Taxes and Licenses
...,...,...,...,...
507,Unrealized Gains/(Losses) - Foreign Currency,,,
508,Vat Output - Others,VAT Payable,Expense,Accrued Expenses and Other Liabilities
509,Withdrawal-Interest,,,
510,Withdrawal-Principal,,,


In [60]:
def merge_fs_classification(
    tb_fs_df: pd.DataFrame, 
    coa_mapping_df: pd.DataFrame,
    tb_acct_name_col: str = 'TB Account Name',
    acct_type_col: str = 'Account Type',
    fs_classification_col: str = 'FS Classification'
) -> pd.DataFrame:
    """
    Merges the Financial Statement (FS) Classification column from the COA mapping
    into the deduplicated summary table (tb_fs_df), using a robust two-column join.
    
    The join is performed by standardizing (stripping and uppercasing) the values 
    in the key columns across both DataFrames to ensure case-insensitive matching.

    Args:
        tb_fs_df (pd.DataFrame): The target summary DataFrame (e.g., deduplicated 
                                 by TB Account Name and Account Type).
        coa_mapping_df (pd.DataFrame): The source mapping DataFrame containing 
                                       the FS Classification.
        tb_acct_name_col (str): Name of the 'TB Account Name' column in both DFs.
        acct_type_col (str): Name of the 'Account Type' column in both DFs.
        fs_classification_col (str): Name of the target classification column.

    Returns:
        pd.DataFrame: The tb_fs_df with the FS Classification column added.
    """
    print("\n🔗 Starting Two-Column Merge for FS Classification...")

    # Define the keys for merging
    join_keys = [tb_acct_name_col, acct_type_col]
    
    # Calculate the UPPERCASE keys required for indexing the coa_mapping_df 
    # (which has its column names uppercased by merge_coa_details)
    upper_join_keys = [k.upper().strip() for k in join_keys]
    upper_fs_classification_key = fs_classification_col.upper().strip()
    
    # --- 1. Prepare COA Mapping Subset (Source) ---
    try:
        # Select columns from coa_mapping_df using the correct UPPERCASE names
        coa_subset_source = coa_mapping_df[upper_join_keys + [upper_fs_classification_key]].copy()
    except KeyError as e:
        print(f"Error: Could not find required columns in COA Mapping DF. Keys attempted: {upper_join_keys + [upper_fs_classification_key]}.")
        print(f"Available COA columns: {coa_mapping_df.columns.tolist()}")
        print(f"Original Error: {e}")
        return tb_fs_df # Return original DF on error
    
    # Rename the columns back to the mixed-case standard for consistent internal processing 
    # and compatibility with tb_fs_df
    coa_subset = coa_subset_source.rename(columns={
        upper_join_keys[0]: tb_acct_name_col,
        upper_join_keys[1]: acct_type_col,
        upper_fs_classification_key: fs_classification_col
    })
    
    # Create temporary standardized key columns (UPPERCASE VALUE, stripped)
    TEMP_KEY_1 = '__KEY_TB_ACCT'
    TEMP_KEY_2 = '__KEY_ACCT_TYPE'
    
    # Deduplicate the COA subset on the combined key, keeping the first classification found
    # We must create the standardized keys first
    coa_subset[TEMP_KEY_1] = coa_subset[tb_acct_name_col].astype(str).str.strip().str.upper()
    coa_subset[TEMP_KEY_2] = coa_subset[acct_type_col].astype(str).str.strip().str.upper()
    
    coa_merge_source = coa_subset.drop_duplicates(
        subset=[TEMP_KEY_1, TEMP_KEY_2], 
        keep='first'
    )[[TEMP_KEY_1, TEMP_KEY_2, fs_classification_col]]
    
    
    # --- 2. Prepare Target DF (tb_fs_df) ---
    tb_fs_df_out = tb_fs_df.copy()
    
    try:
        # Create the same temporary standardized keys in the target DF
        tb_fs_df_out[TEMP_KEY_1] = tb_fs_df_out[tb_acct_name_col].astype(str).str.strip().str.upper()
        tb_fs_df_out[TEMP_KEY_2] = tb_fs_df_out[acct_type_col].astype(str).str.strip().str.upper()
    except KeyError:
        print(f"Error: Missing key columns in Target DF. Required: {join_keys}")
        return tb_fs_df # Return original DF on error

    # --- 3. Perform Left Merge ---
    merged_df = pd.merge(
        tb_fs_df_out,
        coa_merge_source,
        left_on=[TEMP_KEY_1, TEMP_KEY_2],
        right_on=[TEMP_KEY_1, TEMP_KEY_2],
        how='left'
    )
    
    # --- 4. Cleanup and Reordering ---
    
    # Drop the temporary merge key columns
    merged_df = merged_df.drop(columns=[TEMP_KEY_1, TEMP_KEY_2])
    
    # Reorder columns to put the new classification next to the Account Type
    current_cols = merged_df.columns.tolist()
    
    # Find indices of key columns
    try:
        idx_acct_type = current_cols.index(acct_type_col)
        # Move the new column next to Account Type
        new_cols = current_cols[:]
        if fs_classification_col in new_cols:
             new_cols.remove(fs_classification_col)
             new_cols.insert(idx_acct_type + 1, fs_classification_col)
             merged_df = merged_df[new_cols]
    except ValueError:
        # If column not found, just keep the default order (FS Classification will be last)
        pass 
        
    # --- ADDED: Final Sort by Account Type and then TB Account Name ---
    try:
        # Sort by Account Type first (to group) and then by TB Account Name (for internal alphabetical order)
        merged_df = merged_df.sort_values(by=[acct_type_col, tb_acct_name_col]).reset_index(drop=True)
    except KeyError:
        print(f"Warning: Could not sort by '{acct_type_col}' or '{tb_acct_name_col}'. Sorting skipped.")
    
    print(f"✓ FS Classification merged successfully. Column added: '{fs_classification_col}'")
    return merged_df

In [61]:
def merge_classification_to_detailed_df(
    detailed_df: pd.DataFrame, 
    classified_keys_df: pd.DataFrame, 
    merge_keys: List[str] = ['TB Account Name', 'Account Type'], 
    classification_col: str = 'FS Classification'
) -> pd.DataFrame:
    """
    Performs a left merge to transfer the Financial Statement (FS) Classification 
    from a small key list back onto the large, detailed DataFrame.

    This is necessary when the detailed data source (like the pivot table) is 
    missing the FS classification but already contains the necessary TB Account 
    Name and Account Type columns.

    Args:
        detailed_df (pd.DataFrame): The target DataFrame (e.g., trial_balance_pivot_table) 
                                    that needs the FS Classification column added.
        classified_keys_df (pd.DataFrame): The small, pre-classified DF from a 
                                           previous step (e.g., the output of merge_fs_classification).
        merge_keys (List[str]): The columns used for joining (defaults to 
                                ['TB Account Name', 'Account Type']).
        classification_col (str): The column to be merged (defaults to 'FS Classification').

    Returns:
        pd.DataFrame: The detailed_df with the FS Classification column added.
    """
    print(f"\n🔗 Starting Merge-Back of '{classification_col}' to Detailed DF...")
    
    # 1. Prepare the source: select only the keys and the classification column
    source_cols = merge_keys + [classification_col]
    
    # 2. Drop duplicates in the classified keys DF to ensure a clean many-to-one merge
    source_df = classified_keys_df[source_cols].drop_duplicates(subset=merge_keys, keep='first').copy()
    
    # 3. Perform the merge
    merged_df = pd.merge(
        detailed_df, 
        source_df, 
        on=merge_keys, 
        how='left',
        validate='many_to_one'
    )
    
    print(f"✓ Classification '{classification_col}' successfully merged back into detailed DataFrame.")
    return merged_df

In [62]:
# --- STAGE 4: MERGE FS CLASSIFICATION (User's request) ---
# The new function is called here
tb_fs_df = merge_fs_classification(
    tb_fs_df=tb_fs_df,
    coa_mapping_df=coa_mapping
)

tb_fs_df


🔗 Starting Two-Column Merge for FS Classification...
✓ FS Classification merged successfully. Column added: 'FS Classification'


,TB Account Name,Account Type,FS Classification
0,,,
1,Accounts Receivable,Asset,Loans and receivables
2,Accrued interest Receivable,Asset,Loans and receivables
3,Allowance for Prob. Losses,Asset,Other Expense
4,Cash in Bank,Asset,Cash and Cash Equivalents
5,Dividend Receivables,Asset,Loans and receivables
6,Due from Brokers,Asset,Loans and receivables
7,Due from Intercompany,Asset,Loans and receivables
8,Due from Others,Asset,Loans and receivables
9,Due from PEMI,Asset,Loans and receivables


In [63]:
tb_fs_df.columns

Index(['TB Account Name', 'Account Type', 'FS Classification'], dtype='object')

In [64]:
# --- NEW STAGE 4.5: MERGE CLASSIFICATION BACK TO DETAILED DF (THE MISSING STEP) ---
# We use the new helper function to correct the detailed table.
trial_balance_pivot_table = merge_classification_to_detailed_df(
    detailed_df=trial_balance_pivot_table,
    classified_keys_df=tb_fs_df
)



🔗 Starting Merge-Back of 'FS Classification' to Detailed DF...
✓ Classification 'FS Classification' successfully merged back into detailed DataFrame.


In [65]:
trial_balance_pivot_table

,TB Account Name,Account Type,GL ACCOUNT,BSP,DLS-CSB,DLSU,OLD-GSIS,GSIS,PDYFI,PDIF,PEFI,PEMI,PPBF,PPSE,PAOF,PAOFS,PMPI,IMA 008,SSS,FS Classification
0,Annual Fee - Director,Expense,ANNUAL FEE - DIRECTOR,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Director's Fee
1,BIR Fees,Expense,BIR Registration/License Fee,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-3000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Taxes and Licenses
2,Municipal Permit,Expense,Business Permit,0.00,0.00,0.00,0.00,0.00,0.00,-25.50,-11283543.05,0.00,-0.13,0.00,0.00,0.00,0.00,0.00,0.00,Taxes and Licenses
3,Documentary Stamp Tax,Expense,Doc. Stamp Tax,0.00,0.00,0.00,0.00,0.00,-3373495.28,-0.00,-1358391.92,0.00,-0.12,-0.03,0.00,0.00,0.00,0.00,0.00,Taxes and Licenses
4,Professional Fees,Expense,Legal & Professsional Fees,0.00,0.00,0.00,0.00,0.00,-0.03,-5396.54,6231846.44,0.00,0.00,2426861.40,0.00,0.00,0.00,0.00,0.00,Professional Fee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Cash in Bank,Asset,"Union Bank of the Philippines, Inc.-6101011305",0.00,0.00,0.00,0.00,0.00,0.00,1974222.28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Cash and Cash Equivalents
257,,,Unrealized Gain / Loss,-55505407.93,14759960.46,-43211680.26,0.00,58149555.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,92292705543.68,77587036.34,
258,VAT Payable,Expense,Vat Output - Others,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-6253161246.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Accrued Expenses and Other Liabilities
259,,,Withdrawal-Principal,1530000000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42877500000.00,0.00,


In [66]:
def calculate_summary_totals(
    df: pd.DataFrame, 
    group_cols: List[str] = ['FS Classification', 'Account Type', 'TB Account Name']
) -> pd.DataFrame:
    """
    Aggregates the detailed Trial Balance (or pivot table) by the specified 
    classification columns, summing all numerical columns.

    This generates the final, high-level Financial Statement summary table. 
    It is robust against missing grouping columns by using only the columns 
    that are actually present in the input DataFrame and prevents non-grouping 
    key columns (like GL ACCOUNT) from being erroneously summed.

    Args:
        df (pd.DataFrame): The detailed DataFrame (e.g., final_merged_pivot_table) 
                           containing both classification keys and numerical values.
        group_cols (List[str]): The columns to group by. Defaults to the full 
                                classification hierarchy for the final report.

    Returns:
        pd.DataFrame: A summary table with one row per unique group, containing 
                      the sum of all numerical values.
    """
    print(f"\n📈 Starting Summary Aggregation (Group By: {group_cols})...")

    # 1. Prepare for case-insensitive matching by uppercasing column names and group keys
    
    # Create a copy of the DF and standardize its column names to UPPERCASE
    df_working = df.copy()
    original_to_upper_map = {str(col): str(col).strip().upper() for col in df_working.columns}
    df_working.columns = df_working.columns.map(original_to_upper_map)
    
    # Prepare the list of requested grouping columns in UPPERCASE
    upper_group_cols = [col.upper().strip() for col in group_cols]
    
    # 2. Filter out missing grouping columns
    present_group_cols_upper = [col for col in upper_group_cols if col in df_working.columns]
    missing_cols_upper = [col for col in upper_group_cols if col not in df_working.columns]

    if missing_cols_upper:
        # Find the original mixed-case names for the missing columns to report to the user
        missing_names = [group_cols[upper_group_cols.index(col)] for col in missing_cols_upper]
        print(f"⚠️ WARNING: Missing required grouping columns for aggregation: {missing_names}. Proceeding with aggregation on available columns only.")
    
    if not present_group_cols_upper:
        print("Error: No valid grouping columns found after filtering. Cannot aggregate.")
        return pd.DataFrame()
        
    # 3. CRITICAL FIX: Explicitly drop 'GL ACCOUNT' if it's not a grouping key.
    GL_ACCOUNT_COL = 'GL ACCOUNT'
    if GL_ACCOUNT_COL in df_working.columns and GL_ACCOUNT_COL not in present_group_cols_upper:
        print(f"   Note: Dropping non-grouping key column '{GL_ACCOUNT_COL}' before aggregation to prevent erroneous summation.")
        df_working = df_working.drop(columns=[GL_ACCOUNT_COL])
    
    # 4. Group and sum (pandas automatically sums only the numeric columns)
    summary_df = df_working.groupby(present_group_cols_upper).sum().reset_index()

    # 5. Rename columns back to the mixed-case standard for readability
    
    # Create the rename mapping from UPPERCASE (used for groupby) to MixedCase (desired output)
    rename_dict = {}
    used_mixed_case_cols = []
    
    for upper_col in present_group_cols_upper:
        # Find the original mixed-case name from the requested group_cols list
        index = upper_group_cols.index(upper_col)
        mixed_case_name = group_cols[index]
        rename_dict[upper_col] = mixed_case_name
        used_mixed_case_cols.append(mixed_case_name)

    summary_df = summary_df.rename(columns=rename_dict)
    
    # 6. Sort the final result by the grouping hierarchy (using only the columns present)
    try:
        summary_df = summary_df.sort_values(by=used_mixed_case_cols).reset_index(drop=True)
    except KeyError:
        print("Warning: Could not sort by specified group columns. Sorting skipped.")

    print(f"✓ Summary table created with {len(summary_df)} aggregated rows. Grouped by: {used_mixed_case_cols}")
    return summary_df

In [67]:
# --- STAGE 5: AGGREGATE TO FINAL SUMMARY (Current User Query) ---
tb_fs_df = calculate_summary_totals(
    df=trial_balance_pivot_table,
    group_cols=[ 'TB Account Name', 'Account Type', 'FS Classification']
)
tb_fs_df


📈 Starting Summary Aggregation (Group By: ['TB Account Name', 'Account Type', 'FS Classification'])...
   Note: Dropping non-grouping key column 'GL ACCOUNT' before aggregation to prevent erroneous summation.
✓ Summary table created with 63 aggregated rows. Grouped by: ['TB Account Name', 'Account Type', 'FS Classification']


,TB Account Name,Account Type,FS Classification,BSP,DLS-CSB,DLSU,OLD-GSIS,GSIS,PDYFI,PDIF,PEFI,PEMI,PPBF,PPSE,PAOF,PAOFS,PMPI,IMA 008,SSS
0,,,,7979368348.98,2408728015.04,1512480984.84,4859.93,47936688638.84,58942452256.57,481444.99,212660441284.61,-10149849591.41,3103409442.37,102661618990.68,7213018835.26,-456063.19,8512876934.39,679615718671.58,33262251196.40
1,A/P - Others,Liability,Accrued Expenses and Other Liabilities,0.00,0.00,0.00,384000000.00,0.00,12.10,-535956.66,7999997.88,-6394942320.30,26995499.91,38804.31,0.00,0.00,0.00,0.00,0.00
2,Accounts Receivable,Asset,Loans and receivables,0.00,799.80,0.00,-384051200.40,0.00,3777002.78,2584590.06,-7.29,-13148294050.83,2070000.66,1544996.71,-0.60,-0.30,3164995.20,0.00,0.00
3,Accrued Exp. Payable,Expense,Accrued Expenses and Other Liabilities,0.00,0.00,0.00,0.00,0.00,-55147.06,-849020.67,0.00,0.00,-525168.72,-295.89,0.00,0.00,0.00,0.00,0.00
4,Accrued interest Receivable,Asset,Loans and receivables,160446.38,0.00,0.00,76399973.10,34519.71,558327.00,679353.83,8705206.93,189724975.30,-201188772.31,88771.58,0.06,51587.63,0.00,0.00,0.00
5,Additional Paid-In Capital,Equity,Additional paid-in capital,0.00,0.00,0.00,0.00,0.00,-13190229654.24,-143162902.13,-221702980450.55,0.00,-3609874712.18,-86838471328.74,0.00,0.00,-199408813.94,0.00,0.00
6,Allowance for Prob. Losses,Asset,Other Expense,0.00,0.00,0.00,0.00,0.00,0.00,-0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Annual Fee - Director,Expense,Director's Fee,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,BIR Fees,Expense,Taxes and Licenses,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Bank Charges,Expense,Others,0.00,0.00,0.00,198000.00,0.00,395700.00,48683.67,1253900.00,2674100.00,127500.00,647240.00,341650.00,0.00,325805.00,0.00,9000.00


In [68]:
latest_reporting_dates_flat

{'September': '2025-09-30'}

In [69]:
def create_segmented_dfs(tb_fs_df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    """
    Creates a dictionary of DataFrames, segmenting the input DataFrame
    based on its numerical columns and dropping records where the metric value is zero.

    Each resulting DataFrame contains the three identifying columns 
    ('TB Account Name', 'Account Type', 'FS Classification') as separate
    data columns, followed by a single numerical column.
    
    CRITICAL CHANGE: Records with a 0 value in the numerical column are dropped.

    Args:
        tb_fs_df: The input DataFrame containing all financial and identifying data.

    Returns:
        A dictionary where keys are numerical column names and values are
        the corresponding segmented DataFrames.
    """
    # 1. Define the identifying columns to retain
    IDENTIFIER_COLS: List[str] = ['TB Account Name', 'Account Type', 'FS Classification']

    # 2. Identify the numerical columns based on data type
    numerical_cols: List[str] = tb_fs_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Filter out the identifier columns and known non-financial metrics
    numerical_cols = [col for col in numerical_cols if col not in IDENTIFIER_COLS]
    metrics_to_exclude = ['total_in_out_freq', 'total_out_in_freq', 'total_in_out_duration_hours', 'total_out_in_duration_hours']
    numerical_cols = [col for col in numerical_cols if col not in metrics_to_exclude]

    if not numerical_cols:
        print("Warning: No numerical columns found to segment after exclusion filtering.")
        return {}

    # 3. Create the dictionary of DataFrames
    segmented_dfs: Dict[str, pd.DataFrame] = {}
    
    for num_col in numerical_cols:
        # The list of columns for the new DataFrame
        cols_to_select = IDENTIFIER_COLS + [num_col]

        # Select the columns and create a deep copy
        temp_df = tb_fs_df[cols_to_select].copy()
        
        # 4. CRITICAL STEP: Drop rows where the value in the current numerical column is exactly 0.
        # Note: We use .loc to ensure we modify the copy safely.
        temp_df = temp_df.loc[temp_df[num_col] != 0]

        # Store the filtered DataFrame
        segmented_dfs[num_col] = temp_df

    return segmented_dfs

In [70]:

segmented_results = create_segmented_dfs(tb_fs_df)
print(segmented_results.keys())

# segmented_results['PEMI']

dict_keys(['BSP', 'DLS-CSB', 'DLSU', 'OLD-GSIS', 'GSIS', 'PDYFI', 'PDIF', 'PEFI', 'PEMI', 'PPBF', 'PPSE', 'PAOF', 'PAOFS', 'PMPI', 'IMA 008', 'SSS'])


In [71]:
portfolio_mapping

,level1accountname\t,Fund_Code
0,"PHILEQUITY ALPHA ONE FUND, INC.",PAOF
1,PHILEQUITY DIVIDEND YIELD FUND\t,PDYFI
2,PHILEQUITY DOLLAR INCOME FUND\t,PDIF
3,"PHILEQUITY FUND, INC.\t",PEFI
4,"PHILEQUITY MSCI PHILIPPINES INDEX FUND, INC.\t",PMPI
5,PHILEQUITY PESO BOND FUND\t,PPBF
6,PHILEQUITY PSE INDEX FUND\t,PPSE
7,GOVERNMENT SERVICE INSURANCE SYSTEM-500032,OLD-GSIS
8,"PHILEQUITY MANAGEMENT, INC.\t",PEMI
9,"PHILEQUITY ALPHA ONE FUND, INC. S\t",PAOFS


In [ ]:
def export_financial_report(
    dataframes: Dict[str, pd.DataFrame], 
    segmented_dfs: Dict[str, pd.DataFrame], 
    latest_reporting_dates: Dict[str, str],
    portfolio_mapping: pd.DataFrame,
    # UPDATED FILENAME HERE
    output_filename: str = 'Trial_Balance.xlsx' 
) -> str:
    """
    Exports multiple DataFrames and the dictionary of segmented DataFrames 
    to a single Excel file with specified sheet names and custom headers,
    saving to a dynamic, date-based directory structure (../data/processed/Trail Balance/YYYY/).

    Args:
        dataframes: Dictionary of the initial 5 DataFrames.
        segmented_dfs: Dictionary of DataFrames created by create_segmented_dfs.
        latest_reporting_dates: Dictionary containing the reporting date information.
        portfolio_mapping: DataFrame for fund name lookups.
        output_filename: Name of the Excel file to create.

    Returns:
        The path to the generated Excel file.
    """
    
    # DEBUG: Print input validation
    print(f"DEBUG: export_financial_report called")
    print(f"DEBUG: dataframes keys: {list(dataframes.keys())}")
    print(f"DEBUG: segmented_dfs keys: {list(segmented_dfs.keys())}")
    print(f"DEBUG: output_filename: {output_filename}")
    
    # 1. Prepare data and metadata
    
    # Define the sheet mapping for the initial 5 DFs
    initial_sheet_map = {
        'trial_balance_consolidated': 'RD-TB',
        'chart_of_accounts_final': 'COA',
        'coa_mapping': 'COA_ref',
        'trial_balance_pivot_table': 'TB-Pivot',
        'tb_fs_df': 'TB-SF',
    }
    
    # Extract and format the reporting date
    reporting_date_str = list(latest_reporting_dates.values())[0]
    reporting_date = datetime.strptime(reporting_date_str, '%Y-%m-%d')
    date_formatted = f"as of {reporting_date.strftime('%B %d, %Y')}" 

    # Dynamic Path Creation (IMPLEMENTATION OF USER REQUEST)
    year = reporting_date.strftime('%Y')
    
    # Read output_base_path from config if available, otherwise use default
    default_base_dir = os.path.join('..', 'data', 'processed', 'Trail Balance')
    base_dir = config.get('output_base_path', default_base_dir)
    
    print(f"📂 Output base path: {base_dir}")
    
    # Output directory now only contains the year, omitting the month
    output_dir = os.path.join(base_dir, year)
    output_filepath = os.path.join(output_dir, output_filename)
    
    # Create the directory structure if it doesn't exist
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Directory created/verified: {output_dir}")
    except Exception as e:
        print(f"❌ Error creating directory structure {output_dir}: {e}")
        import traceback
        traceback.print_exc()
        return "Export failed due to directory creation error."

    # --- CRITICAL FIX: Clean up column names by stripping whitespace ---
    portfolio_mapping.columns = portfolio_mapping.columns.str.strip()

    # --- FIX: Check and normalize the 'level1accountname' column casing ---
    target_col_name = 'level1accountname'
    found_col = None
    
    # Iterate through columns to find the case-insensitive match
    for col in portfolio_mapping.columns:
        # Check if the stripped column name, converted to lowercase, matches the target
        if col.lower() == target_col_name:
            found_col = col
            break
            
    if found_col is None:
        # If the column is still not found, raise a clear error indicating what columns were present
        raise KeyError(
            f"The required column '{target_col_name}' (case-insensitive) was not found in 'portfolio_mapping' DataFrame. "
            f"Available columns are: {list(portfolio_mapping.columns)}"
        )
        
    # Standardize portfolio mapping level1accountname (convert to title case for cleaner output)
    # Use the found column name for successful access
    portfolio_mapping['level1accountname_std'] = portfolio_mapping[found_col].str.title()
    portfolio_mapping_dict = portfolio_mapping.set_index('Fund_Code')['level1accountname_std'].to_dict()

    # Define the starting row index for the DataFrame's column names (Excel Row 5).
    # This pushes the data to start on Excel Row 6, inserting a blank row (Row 4) 
    # between the date header (Row 3) and the column names (Row 5).
    HEADER_ROW_EXCEL_NUM = 5 
    
    # Define the standard number format: Thousand separator and 2 decimal places
    NUMBER_FORMAT = '#,##0.00' 
    TOTAL_COLUMN_EXCEL_INDEX = 4 # The numerical value is in the 4th column (Excel column index 4)
    # Define the bold font style
    BOLD_FONT = Font(bold=True)

    # 2. Initialize Excel Writer with openpyxl engine
    try:
        print(f"Creating Excel file: {output_filepath}")
        # Use openpyxl engine and ensure the workbook object is accessible
        writer = pd.ExcelWriter(output_filepath, engine='openpyxl')
        
        # 3. Write the initial 5 DataFrames
        print(f"Writing initial 5 DataFrames...")
        for df_key, sheet_name in initial_sheet_map.items():
            if df_key in dataframes:
                print(f"  Writing sheet: {sheet_name} ({len(dataframes[df_key])} rows)")
                dataframes[df_key].to_excel(writer, sheet_name=sheet_name, index=False)
                # Note: Default formatting is not applied to these sheets as they are raw dumps
            else:
                print(f"⚠️ Warning: DataFrame '{df_key}' not found.")
                
        # 4. Write the segmented DataFrames with custom headers and formatting
        print(f"Writing {len(segmented_dfs)} segmented DataFrames...")
        for num_col, df in segmented_dfs.items():
            print(f"  Writing sheet: {num_col} ({len(df)} rows)")
            sheet_name = num_col 
            
            # Look up the fund name. Use the dictionary key (num_col) for lookup.
            fund_name = portfolio_mapping_dict.get(
                num_col, 
                "Portfolio: Not Mapped"
            )
            
            # The list of header values for the first column (Rows 1, 2, 3)
            custom_header = [
                fund_name,             # Row 1, Col A: Level 1 Account Name
                "Trial Balance",       # Row 2, Col A: Fixed string
                date_formatted         # Row 3, Col A: As of Date
            ]

            # Write the DataFrame starting at the correct row index (Row 6)
            df.to_excel(writer, sheet_name=sheet_name, startrow=HEADER_ROW_EXCEL_NUM, header=False, index=False)
            
            # Access the openpyxl workbook and worksheet to write the custom headers
            worksheet = writer.sheets[sheet_name]
            
            # --- NEW IMPLEMENTATION: Merge and Bold Header Rows (1, 2, 3) ---
            for row_index, value in enumerate(custom_header):
                row_num_excel = row_index + 1
                
                # 1. Write the custom header value to the first cell (A1, A2, A3)
                cell = worksheet.cell(row=row_num_excel, column=1, value=value)
                
                # 2. Apply bold font
                cell.font = BOLD_FONT
                
                # 3. Merge cells A and B (columns 1 and 2) for a clean header
                worksheet.merge_cells(start_row=row_num_excel, start_column=1, end_row=row_num_excel, end_column=2)
            # --- END NEW HEADER IMPLEMENTATION ---
                
            # Write the DataFrame header (column names) above the data to Row 5
            for col_num, column_name in enumerate(df.columns):
                worksheet.cell(row=HEADER_ROW_EXCEL_NUM, column=col_num + 1, value=column_name)
                

            # --- NEW: Apply 2-decimal formatting to the numerical column (Column 4) ---
            # Calculate data row boundaries
            data_start_row_excel = HEADER_ROW_EXCEL_NUM + 1 # i.e., Excel row 6
            data_end_row_excel = data_start_row_excel + len(df)
            
            for row_num in range(data_start_row_excel, data_end_row_excel):
                cell = worksheet.cell(row=row_num, column=TOTAL_COLUMN_EXCEL_INDEX)
                # Setting the number format for the data cells
                cell.number_format = NUMBER_FORMAT

            # --- NEW: Grand Total calculation and writing (skip 2 rows) ---
            if not df.empty:
                numerical_col_name = df.columns[-1]
                # Safely calculate grand total
                grand_total = pd.to_numeric(df[numerical_col_name], errors='coerce').fillna(0).sum()
                
                # Calculate the Excel row number for the total (Last data row + 2)
                total_row_num = data_end_row_excel + 2
                
                # 1. Write the 'Grand Total' label in Column C (Column index 3)
                label_cell = worksheet.cell(row=total_row_num, column=TOTAL_COLUMN_EXCEL_INDEX - 1, value="Grand Total")
                
                # 2. Apply Right Alignment to the label cell
                label_cell.alignment = Alignment(horizontal='right')
                # 3. Make the label bold
                label_cell.font = Font(bold=True)
                
                # Write the calculated sum in the numerical column (Column index 4) and APPLY FORMAT
                total_cell = worksheet.cell(row=total_row_num, column=TOTAL_COLUMN_EXCEL_INDEX, value=grand_total)
                total_cell.number_format = NUMBER_FORMAT # Apply the 2-decimal format
            # --- END NEW IMPLEMENTATION ---

        # 5. Save the file
        print(f"Saving Excel file...")
        writer.close()
        print(f"✅ Excel file saved successfully: {output_filepath}")
        
    except Exception as e:
        print(f"❌ ERROR: An error occurred during Excel export: {e}")
        import traceback
        traceback.print_exc()

        return "Export failed."

    return output_filepath


In [73]:
portfolio_mapping

,level1accountname\t,Fund_Code
0,"PHILEQUITY ALPHA ONE FUND, INC.",PAOF
1,PHILEQUITY DIVIDEND YIELD FUND\t,PDYFI
2,PHILEQUITY DOLLAR INCOME FUND\t,PDIF
3,"PHILEQUITY FUND, INC.\t",PEFI
4,"PHILEQUITY MSCI PHILIPPINES INDEX FUND, INC.\t",PMPI
5,PHILEQUITY PESO BOND FUND\t,PPBF
6,PHILEQUITY PSE INDEX FUND\t,PPSE
7,GOVERNMENT SERVICE INSURANCE SYSTEM-500032,OLD-GSIS
8,"PHILEQUITY MANAGEMENT, INC.\t",PEMI
9,"PHILEQUITY ALPHA ONE FUND, INC. S\t",PAOFS


In [88]:


# 3. Prepare the dictionary of the 5 main DFs explicitly from the unpacked variables
# This dictionary matches the expected input structure for the export function.
main_dfs = {
    'trial_balance_consolidated': trial_balance_consolidated, 
    'chart_of_accounts_final': chart_of_accounts_final, 
    'coa_mapping': coa_mapping, 
    'trial_balance_pivot_table': trial_balance_pivot_table, 
    'tb_fs_df': tb_fs_df
}

# 4. Export the final report
output_file = export_financial_report(
    dataframes=main_dfs,
    segmented_dfs=segmented_results,
    latest_reporting_dates=latest_reporting_dates_flat,
    portfolio_mapping=portfolio_mapping,
    # Passing the new desired filename explicitly
    output_filename='Trial_Balance.xlsx' 
)

DEBUG: export_financial_report called
DEBUG: dataframes keys: ['trial_balance_consolidated', 'chart_of_accounts_final', 'coa_mapping', 'trial_balance_pivot_table', 'tb_fs_df']
DEBUG: segmented_dfs keys: ['BSP', 'DLS-CSB', 'DLSU', 'OLD-GSIS', 'GSIS', 'PDYFI', 'PDIF', 'PEFI', 'PEMI', 'PPBF', 'PPSE', 'PAOF', 'PAOFS', 'PMPI', 'IMA 008', 'SSS']
DEBUG: output_filename: Trial_Balance.xlsx
Directory created/verified: ..\data\processed\Trail Balance\2025
Creating Excel file: ..\data\processed\Trail Balance\2025\Trial_Balance.xlsx
Writing initial 5 DataFrames...
  Writing sheet: RD-TB (40162 rows)
  Writing sheet: COA (509 rows)
  Writing sheet: COA_ref (512 rows)
  Writing sheet: TB-Pivot (261 rows)
  Writing sheet: TB-SF (63 rows)
Writing 16 segmented DataFrames...
  Writing sheet: BSP (13 rows)
  Writing sheet: DLS-CSB (15 rows)
  Writing sheet: DLSU (14 rows)
  Writing sheet: OLD-GSIS (21 rows)
  Writing sheet: GSIS (14 rows)
  Writing sheet: PDYFI (43 rows)
  Writing sheet: PDIF (40 rows)
 

In [75]:
tb_fs_df.columns

Index(['TB Account Name', 'Account Type', 'FS Classification', 'BSP',
       'DLS-CSB', 'DLSU', 'OLD-GSIS', 'GSIS', 'PDYFI', 'PDIF', 'PEFI', 'PEMI',
       'PPBF', 'PPSE', 'PAOF', 'PAOFS', 'PMPI', 'IMA 008', 'SSS'],
      dtype='object')

In [76]:
def _get_metric_label_from_key(metric_key: str) -> str:
    """
    Parses a metric key like '2023_Q1_Actual' to return a shorter label like 'Q1 Actual'.
    """
    parts = metric_key.split('_')
    if len(parts) >= 3:
        # Assumes format YYYY_Period_Type (e.g., 2023_Q1_Actual)
        period = parts[-2] 
        metric_type = parts[-1] 
        return f"{period} {metric_type}"
    return metric_key # Fallback

In [ ]:
def export_segmented_summary(
    segmented_dfs: Dict[str, pd.DataFrame], 
    latest_reporting_dates: Dict[str, str],
    portfolio_mapping: pd.DataFrame,
    output_filename: str = 'Segmented_Summary.xlsx' 
) -> str:
    """
    Exports segmented DataFrames to a single Excel file, applying the user's requested
    custom header structure, grand totals, and 2-decimal formatting.

    The data insertion is adjusted to start after 5 rows (on Excel Row 6).
    """
    
    # DEBUG: Print input data
    print(f"DEBUG: segmented_dfs keys: {list(segmented_dfs.keys()) if segmented_dfs else 'EMPTY'}")
    print(f"DEBUG: latest_reporting_dates: {latest_reporting_dates}")
    print(f"DEBUG: output_filename: {output_filename}")
    
    # 1. Prepare metadata
    
    # Extract month name used as the key in the reporting dates dict
    month_name_key = list(latest_reporting_dates.keys())[0]

    # Extract and format the reporting date
    reporting_date_str = list(latest_reporting_dates.values())[0]
    reporting_date = datetime.strptime(reporting_date_str, '%Y-%m-%d')
    date_formatted = f"as of {reporting_date.strftime('%B %d, %Y')}" 

    # Dynamic Path Creation (Uses YYYY only in directory structure)
    year = reporting_date.strftime('%Y')
    
    # Read output_base_path from config if available, otherwise use default
    default_base_dir = os.path.join('..', 'data', 'processed', 'Trail Balance')
    base_dir = config.get('output_base_path', default_base_dir)
    
    print(f"📂 Output base path: {base_dir}")
    
    # The output directory now only contains the year, skipping the month folder
    output_dir = os.path.join(base_dir, year)
    
    output_filepath = os.path.join(output_dir, output_filename)
    
    # Create the directory structure if it doesn't exist
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Directory created/verified: {output_dir}")
    except Exception as e:
        print(f"❌ Error creating directory structure {output_dir}: {e}")
        import traceback
        traceback.print_exc()
        return "Export failed due to directory creation error."
    
    # Clean up column names in portfolio_mapping for robust lookup
    portfolio_mapping.columns = portfolio_mapping.columns.str.strip()
    target_col_name = 'level1accountname'
    found_col = None
    for col in portfolio_mapping.columns:
        if col.lower() == target_col_name:
            found_col = col
            break
            
    if found_col is None:
        raise KeyError(
            f"Required column '{target_col_name}' was not found in 'portfolio_mapping'."
        )
        
    portfolio_mapping['level1accountname_std'] = portfolio_mapping[found_col].str.title()
    portfolio_mapping_dict = portfolio_mapping.set_index('Fund_Code')['level1accountname_std'].to_dict()

    # Define the starting row index for the DataFrame's column names (Excel Row 5).
    HEADER_ROW_EXCEL_NUM = 5 
    DATA_START_ROW = HEADER_ROW_EXCEL_NUM + 1 # For df.to_excel startrow index

    # Define the standard number format and column index for numerical data
    NUMBER_FORMAT = '#,##0.00' 
    TOTAL_COLUMN_EXCEL_INDEX = 4 # The numerical value is in the 4th column (Excel column index 4)
    # Define the bold font style
    BOLD_FONT = Font(bold=True)

    # 2. Initialize Excel Writer with openpyxl engine
    try:
        print(f"Creating Excel file at: {output_filepath}")
        writer = pd.ExcelWriter(output_filepath, engine='openpyxl')
        
        # 3. Write the segmented DataFrames with custom headers
        for num_col, df in segmented_dfs.items():
            print(f"  Writing sheet: {num_col} ({len(df)} rows)")
            sheet_name = num_col 
            
            # Look up the fund name
            fund_name = portfolio_mapping_dict.get(
                num_col, 
                f"Portfolio: {num_col}"
            )
            
            # The list of header values for the first column (Rows 1, 2, 3)
            custom_header = [
                fund_name,             # Row 1, Col A
                "Trial Balance Summary", # Row 2, Col A
                date_formatted         # Row 3, Col A
            ]

            # Use the month name as the 4th column header
            value_col_header = month_name_key 

            # Write the DataFrame data (without headers) starting at Row 6
            df.to_excel(writer, sheet_name=sheet_name, startrow=DATA_START_ROW, header=False, index=False)
            
            # Access the openpyxl workbook and worksheet to write the custom headers
            worksheet = writer.sheets[sheet_name]
            
            # --- NEW IMPLEMENTATION: Merge and Bold Header Rows (1, 2, 3) ---
            for row_index, value in enumerate(custom_header):
                row_num_excel = row_index + 1
                
                # 1. Write the custom header value to the first cell (A1, A2, A3)
                cell = worksheet.cell(row=row_num_excel, column=1, value=value)
                
                # 2. Apply bold font
                cell.font = BOLD_FONT
                
                # 3. Merge cells A and B (columns 1 and 2) for a clean header
                worksheet.merge_cells(start_row=row_num_excel, start_column=1, end_row=row_num_excel, end_column=2)
            # --- END NEW HEADER IMPLEMENTATION ---
            
            # # Write the custom headers to Column A (column index 1)
            # for row_index, value in enumerate(custom_header):
            #     # Excel rows are 1-based, custom_header elements go into rows 1, 2, 3
            #     worksheet.cell(row=row_index + 1, column=1, value=value)
                
            # Write the DataFrame header (column names) to Row 5 (index 4)
            # The identifiers: 'TB Account Name', 'Account Type', 'FS Classification'
            for col_num, column_name in enumerate(df.columns[:3]):
                worksheet.cell(row=HEADER_ROW_EXCEL_NUM, column=col_num + 1, value=column_name)

            # Write the custom metric header (e.g., 'September') to Row 5, Column 4
            worksheet.cell(
                row=HEADER_ROW_EXCEL_NUM, 
                column=4, 
                value=value_col_header
            )
            
            # --- START NEW IMPLEMENTATION: Formatting and Grand Total ---
            
            # 1. Apply 2-decimal formatting to the numerical column (Column 4)
            
            # Calculate data row boundaries
            data_start_row_excel = DATA_START_ROW # i.e., Excel row 6
            data_end_row_excel = data_start_row_excel + len(df)
            
            for row_num in range(data_start_row_excel, data_end_row_excel):
                cell = worksheet.cell(row=row_num, column=TOTAL_COLUMN_EXCEL_INDEX)
                # Setting the number format for the data cells
                cell.number_format = NUMBER_FORMAT

            # 2. Grand Total calculation and writing (skip 2 rows)
            if not df.empty:
                numerical_col_name = df.columns[-1]
                # Safely calculate grand total
                grand_total = pd.to_numeric(df[numerical_col_name], errors='coerce').fillna(0).sum()
                
                # Calculate the Excel row number for the total (Last data row + 2)
                total_row_num = data_end_row_excel + 2
                
                # 1. Write the 'Grand Total' label in Column C (Column index 3)
                label_cell = worksheet.cell(row=total_row_num, column=TOTAL_COLUMN_EXCEL_INDEX - 1, value="Grand Total")
                
                # 2. Apply Right Alignment to the label cell
                label_cell.alignment = Alignment(horizontal='right')
                # 3. Make the label bold
                label_cell.font = Font(bold=True)
                
                # Write the 'Grand Total' label in Column A (Column index 1)
                worksheet.cell(row=total_row_num, column=TOTAL_COLUMN_EXCEL_INDEX - 1, value="Grand Total")
                
                # Write the calculated sum in the numerical column (Column index 4) and APPLY FORMAT
                total_cell = worksheet.cell(row=total_row_num, column=TOTAL_COLUMN_EXCEL_INDEX, value=grand_total)
                total_cell.number_format = NUMBER_FORMAT # Apply the 2-decimal format
            # --- END NEW IMPLEMENTATION ---


        # 4. Save the file
        print(f"Saving Excel file...")
        writer.close()
        print(f"✅ Excel file saved successfully: {output_filepath}")
        
    except Exception as e:
        print(f"❌ ERROR: An error occurred during Excel export: {e}")
        import traceback
        traceback.print_exc()

        return "Export failed."

    return output_filepath


In [86]:
# 3. Export the new segmented summary report
output_file = export_segmented_summary(
    segmented_dfs=segmented_results,
    latest_reporting_dates=latest_reporting_dates_flat,
    portfolio_mapping=portfolio_mapping,
    # UPDATED FILENAME HERE
    output_filename='Trial Balance Monthly.xlsx' 
)


DEBUG: segmented_dfs keys: ['BSP', 'DLS-CSB', 'DLSU', 'OLD-GSIS', 'GSIS', 'PDYFI', 'PDIF', 'PEFI', 'PEMI', 'PPBF', 'PPSE', 'PAOF', 'PAOFS', 'PMPI', 'IMA 008', 'SSS']
DEBUG: latest_reporting_dates: {'September': '2025-09-30'}
DEBUG: output_filename: Trial Balance Monthly.xlsx
Directory created/verified: ..\data\processed\Trail Balance\2025
Creating Excel file at: ..\data\processed\Trail Balance\2025\Trial Balance Monthly.xlsx
  Writing sheet: BSP (13 rows)
  Writing sheet: DLS-CSB (15 rows)
  Writing sheet: DLSU (14 rows)
  Writing sheet: OLD-GSIS (21 rows)
  Writing sheet: GSIS (14 rows)
  Writing sheet: PDYFI (43 rows)
  Writing sheet: PDIF (40 rows)
  Writing sheet: PEFI (43 rows)
  Writing sheet: PEMI (17 rows)
  Writing sheet: PPBF (37 rows)
  Writing sheet: PPSE (39 rows)
  Writing sheet: PAOF (29 rows)
  Writing sheet: PAOFS (12 rows)
  Writing sheet: PMPI (36 rows)
  Writing sheet: IMA 008 (14 rows)
  Writing sheet: SSS (10 rows)
Saving Excel file...
✅ Excel file saved successfu

In [82]:
# fix file base on year folder -> for naming convention if on month data on the file use num val for identification i,e january = TB_2025_1, if january and feb = TB_2025_1-2, 
# if jan to jul = TB_{year}_1-6 = TB_{year}_{month_num_range} 




## 10. Automation Workflow - [Next Steps]

In [80]:
# TODO: Add automation logic here
# - Validation
# - Reconciliation
# - Report generation
# - Export processed data

print("Ready for automation workflow implementation")

Ready for automation workflow implementation
